# Data Analysis

#### Setup

In [ ]:
import os
import sys
import re

import time
import datetime
import difflib
import warnings

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import cm 
from collections import Counter
from sklearn.cluster import KMeans
from scipy import stats

sns.reset_orig()

In [ ]:
path_base = os.path.dirname((os.getcwd())) # Base path of the repository
sys.path.append(path_base) # Add base path to system path

from utils.time import time_difference

In [ ]:
# # Pandas display options
# pd.set_option("display.float_format", "{:.2f}".format)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", 50)
# pd.set_option("display.width", 1000)

# # First reset to default matplotlib style
# plt.rcdefaults()

# # Define a consistent color palette (using matplotlib default colors)
# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Default figure parameters
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "figure.dpi": 300,
    "figure.autolayout": True,
    # Font properties
    "font.size": 11,
    "axes.titlesize": 12,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    # Axes appearance
    # "axes.grid": True,
    "grid.alpha": 0.5,
    # "grid.linestyle": "--",
    # Edge color and linewidth for patches
    "patch.edgecolor": "black",
    "patch.linewidth": 0.75,
    "axes.linewidth": 1.0,
})

# Seaborn style settings
sns.set_style({"patch.edgecolor": "black", "patch.linewidth": 0.75})

warnings.filterwarnings("ignore")

In [ ]:
path_data = os.path.join(path_base, "data", "raw") # Path to the CSV files

In [ ]:
# Status columns to be parsed as datetime
status_columns = ["ASSIGNED", "READY", "WORKING", "DELIVERED", "RECEIVED", "CLOSE"]

In [ ]:
# Load CSVs
try: 
    df_clients = pd.read_csv(os.path.join(path_data, "clients.csv"))
    df_data = pd.read_csv(os.path.join(path_data, "data.csv"), dtype={"PROJECT_ID": "object"}, parse_dates=status_columns)
    df_schedules = pd.read_csv(os.path.join(path_data, "schedules.csv"))
    df_translators = pd.read_csv(os.path.join(path_data, "translatorsCostPairs.csv"))
except FileNotFoundError as e:
    print(f"File not found: {e}")

#### Overview

In [ ]:
tuples_df = {
    "Clients": df_clients,
    "Sample": df_data,
    "Schedules": df_schedules,
    "Translators": df_translators
}

In [ ]:
print("MISSING VALUES\n")

for name, df in tuples_df.items():
    print(f"{name} Dataset")
    print(f"{df.isnull().sum().sum()} missing values\n")

In [ ]:
print("DUPLICATED VALUES\n")

for name, df in tuples_df.items():
    print(f"{name} Dataset")
    print(f"  {df.duplicated().sum()} duplicated value\n")

In [ ]:
print("UNIQUE VALUES\n")

for name, df in tuples_df.items():
    print(f"{name} Dataset")
    print(f"  Unique Values:")
    for col in df.columns:
        print(f"    {col}: {df[col].nunique()}")    
    print()

In [ ]:
print("DATASETS\n")

for name, df in tuples_df.items():
    print(f"{name} Dataset")
    print(f"  {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"  Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"    {col}: {dtype}n")
    print()

In [ ]:
def get_unique_values(df):
    """
    Get unique values for each column in a DataFrame

    Parameters:
        df (pd.DataFrame): DataFrame to get unique values from

    Returns:
        dict: Unique values for each column
    """

    unique_values = {}
    for col in df.columns:
        unique_values[col] = df[col].unique()
    return unique_values


In [ ]:
cols_useful = {
    "Clients": ["CLIENT_NAME", "WILDCARD"],
    "Sample": [
        "TASK_TYPE",
        "SOURCE_LANG",
        "TARGET_LANG",
        "TRANSLATOR",
        "MANUFACTURER",
        "MANUFACTURER_SECTOR",
        "MANUFACTURER_INDUSTRY_GROUP",
        "MANUFACTURER_INDUSTRY",
        "MANUFACTURER_SUBINDUSTRY",
    ],
    "Schedules": ["NAME"],
    "Translators": ["TRANSLATOR", "SOURCE_LANG", "TARGET_LANG", "HOURLY_RATE"],
}

In [ ]:
print("UNIQUE VALUES\n")

# Unique values for each column in each DataFrame
unique_values = {}

for name, df in tuples_df.items():
    unique_values[name] = get_unique_values(df[cols_useful[name]])   

for name, values in unique_values.items():
    print(f"{name} Dataset")
    for col, unique_vals in values.items():
        display(pd.DataFrame({col: unique_vals}))
    print()

#### CSV Clients

In [ ]:
print(f"Number of columns: {df_clients.shape[1]}")
print(f"Number of unique clients: {df_clients['CLIENT_NAME'].nunique()}")

In [ ]:
print("Sample data:")
display(df_clients.head())

In [ ]:
# Summary statistics with additional metrics skewness and kurtosis
print("Detailed summary statistics:")
numeric_cols = df_clients.select_dtypes(include=["number"])  # Select only numeric columns
client_stats = numeric_cols.describe()
client_stats.loc["skew"] = numeric_cols.skew()
client_stats.loc["kurtosis"] = numeric_cols.kurtosis()

display(client_stats)

In [ ]:
# Number of unique clients
print(f"Number of unique clients: {df_clients['CLIENT_NAME'].nunique()}")

In [ ]:
# Check for duplicates in client names
duplicate_clients = df_clients[df_clients.duplicated(subset=["CLIENT_NAME"], keep=False)]
print(f"Number of duplicate client names: {len(duplicate_clients)}")
if len(duplicate_clients) > 0:
    print("Duplicate clients found:")
    display(duplicate_clients)

In [ ]:
# Distribution of selling hourly prices
plt.figure(figsize=(14, 8))
sns.histplot(df_clients["SELLING_HOURLY_PRICE"], kde=True, bins=20, alpha=0.75)
plt.title("Distribution of Selling Hourly Prices")
plt.xlabel("Price")
plt.ylabel("Count Clients")
plt.tight_layout()
plt.show()

# Distribution of Minimum Quality requirements
plt.figure(figsize=(14, 8))
sns.histplot(df_clients["MIN_QUALITY"], kde=True, bins=20, alpha=0.75)
plt.title("Distribution of Minimum Quality Requirements")
plt.xlabel("Minimum Quality")
plt.ylabel("Count Clients")
plt.tight_layout()
plt.show()

In [ ]:
# Analyze relationship between selling price and Minimum Quality
plt.figure(figsize=(14, 8))
sns.scatterplot(data=df_clients, x="MIN_QUALITY", y="SELLING_HOURLY_PRICE")
plt.title("Relationship between Minimum Quality and Selling Price")
plt.xlabel("Minimum Quality")
plt.ylabel("Selling Hourly Price")
plt.tight_layout()
plt.show()

In [ ]:
# Average selling price by Minimum Quality
avg_price_by_quality = df_clients.groupby("MIN_QUALITY")["SELLING_HOURLY_PRICE"].mean().reset_index()

print("Average selling price by Minimum Quality requirement:")
for quality in avg_price_by_quality["MIN_QUALITY"]:
    print(f"Quality {quality:.0f}: {avg_price_by_quality[avg_price_by_quality['MIN_QUALITY'] == quality]['SELLING_HOURLY_PRICE'].values[0]:.2f}")

plt.figure(figsize=(14, 8))
ax = sns.barplot(data=avg_price_by_quality, x="MIN_QUALITY", y="SELLING_HOURLY_PRICE", alpha=0.75)
plt.title("Average Selling Price by Minimum Quality Requirement")
plt.xlabel("Minimum Quality Requirement")
plt.ylabel("Average Selling Hourly Price")

# Add count labels
for i, p in enumerate(ax.patches):
    ax.text(
        p.get_x() + p.get_width() / 2,
        p.get_height(),
        f"{avg_price_by_quality['SELLING_HOURLY_PRICE'].iloc[i]:.2f}",
        ha="center",
        va="bottom",
    )

plt.tight_layout()
plt.show()

In [ ]:
# WILDCARD analysis
wildcard_counts = df_clients["WILDCARD"].value_counts()

# Frquency of WILDCARD values
print("Frequency of WILDCARD values:")
for wildcard, count in wildcard_counts.items():
    print(f"WILDCARD {wildcard}: {count} occurrences")


plt.figure(figsize=(14, 8))
plt.bar(wildcard_counts.index, wildcard_counts.values, alpha=0.75)
plt.title("Frequency of WILDCARD Values")
plt.xlabel("WILDCARD Value")
plt.ylabel("Count")

# Add count labels
for i, count in enumerate(wildcard_counts.values):
    plt.text(i, count + 0.5, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
price_quantiles = df_clients["SELLING_HOURLY_PRICE"].quantile([0, 0.25, 0.5, 0.75, 1]).values # Define Pricing Tiers based on Quantiles
price_quantiles = pd.unique(price_quantiles) # Avoid duplicates in quantiles
pricing_tier_labels = ["Budget", "Economy", "Standard", "Premium", "Enterprise"]

pricing_tiers = pd.cut( # Assign Pricing Tiers based on hourly selling price
    df_clients["SELLING_HOURLY_PRICE"], 
    bins=price_quantiles, 
    labels=pricing_tier_labels[:len(price_quantiles)-1],  # Adjust labels to match unique bins
    include_lowest=True
)
df_clients["PRICING_TIER"] = pricing_tiers

# Calculate Pricing Tier distribution
tier_counts = df_clients["PRICING_TIER"].value_counts().sort_index()
tier_percentages = tier_counts / len(df_clients) * 100
# print(tier_counts)

print("Pricing Tier distribution:")
for tier, count in tier_counts.items():
    percentage = tier_percentages[tier]
    price_range = f"{price_quantiles[pricing_tier_labels.index(tier)]:.2f} - {price_quantiles[pricing_tier_labels.index(tier) + 1]:.2f}"
    print(f"  {tier} ({price_range}): {count} clients ({percentage:.2f}%)")

# Visualize Pricing Tiers
plt.figure(figsize=(14, 8))
plt.bar(tier_counts.index, tier_counts.values, alpha=0.75)
plt.title("Client distribution by Pricing Tier")
plt.xlabel("Pricing Tier")
plt.ylabel("Number Clients")

# Add count labels
for i, count in enumerate(tier_counts.values):
    plt.text(i, count + 0.5, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Quality Tiers
quality_tier_bounds = [0, 6, 7, 8, 9, 10]
quality_tier_labels = ["Minimal", "Basic", "Standard", "High", "Premium"]

# Assign Quality Tiers based on Minimum Quality requirement
quality_tiers = pd.cut(
    df_clients["MIN_QUALITY"], 
    bins=quality_tier_bounds, 
    labels=quality_tier_labels,
    include_lowest=True
)

df_clients["QUALITY_TIER"] = quality_tiers

quality_tier_counts = df_clients["QUALITY_TIER"].value_counts().sort_index()
quality_tier_counts = quality_tier_counts[quality_tier_counts > 0]  # Remove tiers with no clients
quality_tier_percentages = quality_tier_counts / len(df_clients) * 100

print("Quality Tier distribution:")
for tier, count in quality_tier_counts.items():
    percentage = quality_tier_percentages[tier]
    tier_index = quality_tier_labels.index(tier)
    quality_range = f"{quality_tier_bounds[tier_index]} - {quality_tier_bounds[tier_index + 1]}"
    print(f"  {tier} ({quality_range}): {count} clients ({percentage:.2f}%)")

# Visualize Quality Tiers
plt.figure(figsize=(14, 8))
plt.bar(quality_tier_counts.index, quality_tier_counts.values, alpha=0.75)
plt.title("Client Distribution by Quality Requirement Tier")
plt.xlabel("Quality Tier")
plt.ylabel("Number Clients")

# Add count labels
for i, count in enumerate(quality_tier_counts.values):
    plt.text(i, count + 0.5, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Cross-tabulation of price tiers vs. Quality Tiers
price_quality_crosstab = pd.crosstab(
    df_clients["PRICING_TIER"], 
    df_clients["QUALITY_TIER"],
    normalize="index"
) * 100

print("Relationship between Pricing and Quality Tiers:")
print(price_quality_crosstab)

# Create a heatmap for the cross-tabulation
plt.figure(figsize=(14, 8))
sns.heatmap(price_quality_crosstab, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title("Pricing Tier vs Quality Tier (% within Pricing Tier)")
plt.ylabel("Pricing Tier")
plt.xlabel("Quality Tier")
plt.tight_layout()
plt.show()

# Think about this as:
# - Percentage distribution of Quality Tiers within each Pricing Tier [Minimal | Basic | Standard] highlighting relationships between client pricing and quality preferences"
# - e.g, 41.2% of "Economy" clients have "Standard" quality requirements
# - e.g, 39.7% of "Budget" clients have "Basic" quality requirements

In [ ]:
# How much a client pays per unit of their Minimum Quality requirement
# - Low value: They pay less per quality point, potentially a "better deal"
# - High value: They pay more per quality point, higher cost for the same quality level 

# Price per quality point
df_clients["PRICE_PER_QUALITY"] = df_clients["SELLING_HOURLY_PRICE"] / df_clients["MIN_QUALITY"]

# Display price per quality point statistics
print("Price per Quality Point statistics:")
price_per_quality_stats = df_clients[~df_clients["PRICE_PER_QUALITY"].isin([np.inf])]["PRICE_PER_QUALITY"].describe()
print(price_per_quality_stats)

# Visualize price per quality distribution
plt.figure(figsize=(14, 8))
sns.histplot(df_clients["PRICE_PER_QUALITY"], kde=True, bins=25, alpha=0.75)
plt.title("Distribution of Price per Quality Point")
plt.xlabel("Price per Quality Point")
plt.ylabel("Count")
plt.axvline(price_per_quality_stats["mean"], color="blue", linestyle="--", label=f"Mean: {price_per_quality_stats['mean']:.2f}")
plt.axvline(price_per_quality_stats["50%"], color="red", linestyle="--", label=f"Median: {price_per_quality_stats['50%']:.2f}")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Relationship between WILDCARD and selling price
print("Average selling price by WILDCARD:")
print(df_clients.groupby("WILDCARD")["SELLING_HOURLY_PRICE"].mean())

In [ ]:
# Relationship between WILDCARD and Minimum Quality
print("Average Minimum Quality by WILDCARD:")
print(df_clients.groupby("WILDCARD")["MIN_QUALITY"].mean())

In [ ]:
# Analyze WILDCARD field more deeply
print("WILDCARD representation:")
wildcard_counts = df_clients["WILDCARD"].value_counts()
wildcard_percentage = wildcard_counts / len(df_clients) * 100

for wildcard, count in wildcard_counts.items():
    percentage = wildcard_percentage[wildcard]
    print(f"  {wildcard}: {count} clients ({percentage:.2f}%)")

# Cross-tabulate WILDCARD vs Quality Tier
wildcard_quality_crosstab = pd.crosstab(
    df_clients["WILDCARD"], 
    df_clients["QUALITY_TIER"],
    normalize="index"
) * 100

# Cross-tabulate WILDCARD vs Pricing Tier
wildcard_price_crosstab = pd.crosstab(
    df_clients["WILDCARD"], 
    df_clients["PRICING_TIER"],
    normalize="index"
) * 100

print("\nRelationship between WILDCARD and Quality Tier:")
# Visualize relationship between WILDCARD and Quality Tier
plt.figure(figsize=(14, 8))
sns.heatmap(wildcard_quality_crosstab, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("WILDCARD vs Quality Tier")
plt.ylabel("WILDCARD")
plt.xlabel("Quality Tier")
plt.tight_layout()
plt.show()

print("\nRelationship between WILDCARD and Pricing Tier:")
# Visualize relationship between WILDCARD and Pricing Tier
plt.figure(figsize=(14, 8))
sns.heatmap(wildcard_price_crosstab, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("WILDCARD vs Pricing Tier")
plt.ylabel("WILDCARD")
plt.xlabel("Pricing Tier")
plt.tight_layout()
plt.show()


# Think about this as:
# - Percentage distribution of quality/Pricing Tiers within each WILDCARD, highlighting relationships between client preferences and quality/pricing
# - e.g., 31.9% of "Deadline" WILDCARD clients have "Minimal" quality requirements
# - e.g., 39.7% of "Deadline" WILDCARD clients have "Budget" pricing requirements

##### Clients Segments

In [ ]:
# Analysis of clients based on price and quality
# - Use KMeans clustering to group clients based on selling price and Minimum Quality requirements
# - Analyze the clusters to identify different client segments

# Prepare data for clustering
cluster_data = df_clients[["SELLING_HOURLY_PRICE", "MIN_QUALITY"]].copy()

# Standardize data for KMeans
cluster_data_scaled = (cluster_data - cluster_data.mean()) / cluster_data.std()
# print(cluster_data.head())
# print(cluster_data_scaled.head())

# Determine optimal number of clusters
inertia = []
k_range = range(1, 10)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(cluster_data_scaled)
    inertia.append(kmeans.inertia_)

# Plot elbow method
plt.figure(figsize=(14, 8))
plt.plot(k_range, inertia, marker="o")
plt.title("Elbow Method")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.tight_layout()
plt.show()


# Choose optimal k
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df_clients["CLUSTER"] = kmeans.fit_predict(cluster_data_scaled)

# Analyze clusters
cluster_stats = df_clients.groupby("CLUSTER").agg({
    "SELLING_HOURLY_PRICE": ["mean", "min", "max", "std", "count"],
    "MIN_QUALITY": ["mean", "min", "max", "std"],
    "PRICE_PER_QUALITY": ["mean", "min", "max", "std"],
    "WILDCARD": lambda x: x.value_counts().index[0]  # Most common WILDCARD
}).round(2)

print("Client cluster statistics:")
display(cluster_stats.T)

In [ ]:
print("Cluster distribution:")
cluster_counts = df_clients["CLUSTER"].value_counts().sort_index()
cluster_percentages = cluster_counts / len(df_clients) * 100

for cluster, count in cluster_counts.items():
    percentage = cluster_percentages[cluster]
    print(f"  Cluster {cluster}: {count} clients ({percentage:.2f}%)")

# Visualize clusters
plt.figure(figsize=(14, 8))
scatter = plt.scatter(
    df_clients["SELLING_HOURLY_PRICE"], 
    df_clients["MIN_QUALITY"],
    c=df_clients["CLUSTER"], 
    cmap="YlGnBu",
    s=100, 
    alpha=0.75
)
plt.colorbar(scatter, label="Cluster")
plt.title("Client Clusters by Price and Quality")
plt.xlabel("Selling Hourly Price")
plt.ylabel("Minimum Quality Requirement")

# Add cluster centroids
centroids = kmeans.cluster_centers_
centroids_unscaled = centroids * cluster_data.std().values + cluster_data.mean().values
plt.scatter(
    centroids_unscaled[:, 0], 
    centroids_unscaled[:, 1], 
    s=50, 
    marker="x", 
    c="red", 
    label="Centroids"
)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Identify potential client segments based on price sensitivity
price_quality_ratio = df_clients["SELLING_HOURLY_PRICE"] / df_clients["MIN_QUALITY"]

df_clients["PRICE_SENSITIVITY"] = pd.qcut(
    price_quality_ratio, 
    q=3, 
    labels=["High", "Medium", "Low"]
)

print("Client segments by price sensitivity:")
price_sens_counts = df_clients["PRICE_SENSITIVITY"].value_counts().sort_index()
price_sens_percentages = price_sens_counts / len(df_clients) * 100

for sensitivity, count in price_sens_counts.items():
    percentage = price_sens_percentages[sensitivity]
    print(f"- {sensitivity} price sensitivity: {count} clients ({percentage:.2f}%)")

In [ ]:
# Cross-tabulate price sensitivity vs WILDCARD
price_sens_wildcard_crosstab = pd.crosstab(
    df_clients["PRICE_SENSITIVITY"], 
    df_clients["WILDCARD"],
    normalize="index"
) * 100

print("Relationship between price sensitivity and WILDCARD (%):")
display(price_sens_wildcard_crosstab)

In [ ]:
print("KEY FINDINGS FROM CLIENT ANALYSIS:")
print()
print("Total unique clients:")
print(f"    {df_clients['CLIENT_NAME'].nunique()}")
print()
print("Average selling price and price range:")
print(f"    {df_clients['SELLING_HOURLY_PRICE'].mean():.2f}€ ({df_clients['SELLING_HOURLY_PRICE'].min():.2f}€ to {df_clients['SELLING_HOURLY_PRICE'].max():.2f}€)")
print()
print("Average minimum quality requirement:")
print(f"    {df_clients['MIN_QUALITY'].mean():.2f}/10")
print()
print("Distinct client clusters:")
print("    " + str(optimal_k))
print()
print("Client price sensitivity counts:")
print(f"    High: {price_sens_counts['High']}, Medium: {price_sens_counts['Medium']}, Low: {price_sens_counts['Low']}")
print()
print("Most common WILDCARD value:")
print(f"    '{df_clients['WILDCARD'].mode()[0]}' ({wildcard_percentage[df_clients['WILDCARD'].mode()[0]]:.2f}% of clients)")

#### CSV Schedules

In [ ]:
print("Basic information:")
df_schedules.info()

In [ ]:
print("Statistical summary:")

numeric_cols = df_schedules.select_dtypes(include=["number"])  # Select only numeric columns
schedules_stats = numeric_cols.describe()
schedules_stats.loc["skew"] = numeric_cols.skew()
schedules_stats.loc["kurtosis"] = numeric_cols.kurtosis()
display(schedules_stats)

In [ ]:
print("CSV Schedules Data")
display(df_schedules.head())

In [ ]:
allday_cols = ["MON", "TUES", "WED", "THURS", "FRI", "SAT", "SUN"]  # All days of the week
weekday_cols = ["MON", "TUES", "WED", "THURS", "FRI"]               # Weekdays only
weekend_cols = ["SAT", "SUN"]                                       # Weekend days only

In [ ]:
# Apply the time_difference() function (assuming START and END are strings)
df_schedules["HOURS"] = df_schedules.apply(
    lambda row: time_difference(row["START"], row["END"]),
    axis=1
)

total_hours_per_name = df_schedules.groupby("NAME")["HOURS"].sum().reset_index()  # Total available hours per translator

# Display results with better formatting and explicit sorting
print("Total Hours Available per Name:")
display(total_hours_per_name.sort_values(by="HOURS", ascending=True))  # Sort by hours ascending

In [ ]:
# Calculate weekday and weekend hours directly
weekday_hours = df_schedules[weekday_cols].sum(axis=1)
weekend_hours = df_schedules[weekend_cols].sum(axis=1)

# Create a heatmap of weekday vs weekend hours
heatmap_data, x_edges, y_edges = np.histogram2d(weekday_hours, weekend_hours, bins=(6, 3))

# Plot the heatmap
plt.figure(figsize=(14, 8))
plt.imshow(heatmap_data.T, origin="lower", cmap="YlGnBu", aspect="auto",
           extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
plt.colorbar(label="Frequency")
plt.title("Heatmap of Weekday vs Weekend Availability")
plt.xlabel("Weekday Hours")
plt.ylabel("Weekend Hours")
plt.xticks(range(int(x_edges[0]), int(x_edges[-1]) + 1))
plt.yticks(range(int(y_edges[0]), int(y_edges[-1]) + 1))
plt.tight_layout()
plt.show()


In [ ]:
# Calculate maximum availability hours for each translator
df_schedules["DAILY_HOURS"] = df_schedules.apply(
    lambda row: time_difference(row["START"], row["END"]),
    axis=1
)
df_schedules["ACTIVE_DAYS"] = df_schedules[allday_cols].sum(axis=1) # Count active days for each translator
df_schedules["WEEKLY_HOURS"] = df_schedules["DAILY_HOURS"] * df_schedules["ACTIVE_DAYS"] # Calculate total weekly hours

# Weekly hours distribution
plt.figure(figsize=(14, 8))
sns.histplot(df_schedules["WEEKLY_HOURS"], bins=30, kde=True, alpha=0.75)
plt.title("Distribution of Weekly Availability Hours")
plt.xlabel("Weekly Hours")
plt.ylabel("Count Workers")
plt.axvline(df_schedules["WEEKLY_HOURS"].mean(), color="red", linestyle="--", label=f"Mean: {df_schedules['WEEKLY_HOURS'].mean():.2f}")
plt.axvline(df_schedules["WEEKLY_HOURS"].median(), color="blue", linestyle="--", label=f"Median: {df_schedules['WEEKLY_HOURS'].median():.2f}")
plt.legend()
plt.tight_layout()
plt.show()

print("Weekly Availability Hours Statistics:")
print(df_schedules["WEEKLY_HOURS"].describe())

In [ ]:
# Calculate daily averages for each translator (excluding zeros)
df_schedules["AVG_DAILY_HOURS"] = df_schedules[allday_cols].replace(0, np.nan).mean(axis=1, skipna=True)
df_schedules["WORKING_DAYS"] = (df_schedules[allday_cols] > 0).sum(axis=1) # Calculate the number of working days availability per week
df_schedules["HOURS_CONSISTENCY"] = df_schedules[allday_cols].replace(0, np.nan).std(axis=1, skipna=True)

# Identify weekend workers
df_schedules["WEEKEND_WORKER"] = (df_schedules[["SAT", "SUN"]].sum(axis=1) > 0).astype(int)

conditions = [
    (df_schedules["WORKING_DAYS"] >= 5),
    (df_schedules["WORKING_DAYS"] < 5) & (df_schedules["WORKING_DAYS"] >= 3),
    (df_schedules["WORKING_DAYS"] < 3)
]
pattern_values = ["Full-Time", "Part-Time", "Occasional"]
df_schedules["SCHEDULE_PATTERN"] = np.select(conditions, pattern_values, default="Other")

# Display statistics about schedule patterns
pattern_counts = df_schedules["SCHEDULE_PATTERN"].value_counts()
pattern_percentage = pattern_counts / len(df_schedules) * 100

print("Schedule Pattern Distribution:")
for pattern, count in pattern_counts.items():
    percentage = pattern_percentage[pattern]
    print(f"  {pattern}: {count} translators ({percentage:.2f}%)")

# Visualize schedule patterns
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="SCHEDULE_PATTERN", data=df_schedules, 
                  order=pattern_values, alpha=0.75)
plt.title("Distribution of Schedule Patterns")
plt.xlabel("Count")
plt.ylabel("Schedule Pattern")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_schedules)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.tight_layout()
plt.show()


for pattern, group in df_schedules.groupby("SCHEDULE_PATTERN"):
    print(f"\nGroup: {pattern}")
    print(f"Number of translators: {len(group)}")
    display(group[allday_cols + ["WORKING_DAYS"]].head())

In [ ]:
# Calculate the percentage of translators working each day
day_percentage = (df_schedules[allday_cols] > 0).mean() * 100

# Plot percentage of translators working each day
plt.figure(figsize=(14, 8))
ax = sns.barplot(x=day_percentage.index, y=day_percentage.values, alpha=0.75)
plt.title("Percentage of Translators Working by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Percentage Workers")

# Add percentage labels on top of bars
for i, v in enumerate(day_percentage):
    ax.text(i, v + 1, f"{v:.1f}%", ha="center")

# Format y-axis ticks to include percentage signs
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0f}%"))
plt.tight_layout()
plt.show()

In [ ]:
# Extract hours from START and END times
df_schedules["START_HOUR"] = df_schedules["START"].str.split(":", expand=True)[0].astype(int)
df_schedules["END_HOUR"] = df_schedules["END"].str.split(":", expand=True)[0].astype(int)

df_schedules["OVERNIGHT"] = (df_schedules["END_HOUR"] < df_schedules["START_HOUR"]).astype(int) # Account for overnight shifts where END_HOUR < START_HOUR

df_schedules["END_HOUR_ADJ"] = np.where( # Adjust END_HOUR for visualization (add 24 to overnight shift end hours)
    df_schedules["OVERNIGHT"] == 1,
    df_schedules["END_HOUR"] + 24,
    df_schedules["END_HOUR"]
)

# Define work shift categories
conditions = [
    (df_schedules["START_HOUR"] >= 5) & (df_schedules["START_HOUR"] < 9),  # Morning shift
    (df_schedules["START_HOUR"] >= 9) & (df_schedules["START_HOUR"] < 12),  # Mid-morning shift
    (df_schedules["START_HOUR"] >= 12) & (df_schedules["START_HOUR"] < 17),  # Afternoon shift
    (df_schedules["START_HOUR"] >= 17) & (df_schedules["START_HOUR"] < 22),  # Evening shift
    (df_schedules["START_HOUR"] >= 22) | (df_schedules["START_HOUR"] < 5)   # Night shift
]
shift_values = ["Morning", "Mid-Morning", "Afternoon", "Evening", "Night"]
df_schedules["WORK_SHIFT"] = np.select(conditions, shift_values, default="Other")

# Display shift statistics
shift_counts = df_schedules["WORK_SHIFT"].value_counts()
shift_percentage = shift_counts / len(df_schedules) * 100

print("Work Shift Distribution:")
for shift, count in shift_counts.items():
    percentage = shift_percentage[shift]
    print(f"  {shift}: {count} schedules ({percentage:.2f}%)")

# Visualize shift distribution
plt.figure(figsize=(14, 8))
order = ["Morning", "Mid-Morning", "Afternoon", "Evening", "Night", "Other"]
order = [shift for shift in order if shift in shift_counts.index]  # Only include shifts that exist
ax = sns.countplot(y="WORK_SHIFT", data=df_schedules, order=order, alpha=0.75)
plt.title("Distribution of Work Shifts")
plt.xlabel("Count")
plt.ylabel("Work Shift")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_schedules)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.tight_layout()
plt.show()

# Visualize start and end hour distribution
plt.figure(figsize=(16, 10))
plt.subplot(1, 2, 1)
sns.histplot(df_schedules["START_HOUR"], bins=24, kde=True, alpha=0.75)
plt.title("Distribution of Start Hours")
plt.xlabel("Hour of Day (24-hour format)")
plt.ylabel("Count")
plt.xticks(range(0, 24))
plt.grid(True, linestyle="--", alpha=0.5)

plt.subplot(1, 2, 2)
sns.histplot(df_schedules["END_HOUR"], bins=24, kde=True, alpha=0.75)
plt.title("Distribution of End Hours")
plt.xlabel("Hour of Day (24-hour format)")
plt.ylabel("Count")
plt.xticks(range(0, 24))
plt.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Create a matrix of availability by hour and day
hours = range(24)
availability_matrix = np.zeros((24, 7))

for idx, row in df_schedules.iterrows():
    start_hour = row["START_HOUR"]
    end_hour = row["END_HOUR_ADJ"] if row["OVERNIGHT"] == 1 else row["END_HOUR"]
    
    # For each day of the week
    for day_idx, day in enumerate(allday_cols):
        if row[day] > 0: # If the translator works on this day
            for hour in range(start_hour, end_hour): # Mark all hours from start to end as available
                hour_idx = hour % 24  # Wrap around for overnight shifts
                availability_matrix[hour_idx, day_idx] += 1

# Normalize by the total number of translators
availability_percentage = (availability_matrix / len(df_schedules)) * 100

# Visualize the availability heatmap
plt.figure(figsize=(18, 10))
sns.heatmap(availability_percentage, annot=False, cmap="YlGnBu", 
            xticklabels=allday_cols, yticklabels=hours, fmt=".1f")
plt.title("Translator Availability by Hour and Day (%)")
plt.xlabel("Day of Week")
plt.ylabel("Hour of Day")
plt.tight_layout()
plt.show()

# Identify peak and low availability times
hour_availability = availability_percentage.mean(axis=1)  # Average across days
day_availability = availability_percentage.mean(axis=0)   # Average across hours

# Display peak and low availability times
peak_hour = hours[np.argmax(hour_availability)]
low_hour = hours[np.argmin(hour_availability)]
peak_day_idx = np.argmax(day_availability)
low_day_idx = np.argmin(day_availability)

print("Peak and Low Availability:")
print(f"  Peak hour: {peak_hour}:00 with {hour_availability[peak_hour]:.2f}% of translators available")
print(f"  Lowest availability hour: {low_hour}:00 with {hour_availability[low_hour]:.2f}% of translators available")
print(f"  Peak day: {allday_cols[peak_day_idx]} with {day_availability[peak_day_idx]:.2f}% of translators available")
print(f"  Lowest availability day: {allday_cols[low_day_idx]} with {day_availability[low_day_idx]:.2f}% of translators available")

# Visualize hourly and daily availability
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.barplot(x=list(hours), y=hour_availability, alpha=0.75)
plt.title("Average Translator Availability by Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Percentage Available (%)")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)

plt.subplot(1, 2, 2)
sns.barplot(x=allday_cols, y=day_availability, alpha=0.75)
plt.title("Average Translator Availability by Day")
plt.xlabel("Day of Week")
plt.ylabel("Percentage Available (%)")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Define work pattern based on hours
df_schedules["HOURS_WORKED"] = df_schedules.apply(
    lambda row: time_difference(row["START"], row["END"]),
    axis=1
)

df_schedules["WORK_PATTERN"] = pd.cut(
    df_schedules["HOURS_WORKED"],
    bins=[0, 4, 6, 8, 12, 24],
    labels=["Part-time (<4h)", "Part-time (4-6h)", "Standard (6-8h)", "Extended (8-12h)", "Long (>12h)"]
)

work_pattern_counts = df_schedules["WORK_PATTERN"].value_counts().sort_index()
work_pattern_percentage = work_pattern_counts / len(df_schedules) * 100

print("\nWork Pattern Distribution:")
for pattern, count in work_pattern_counts.items():
    percentage = work_pattern_percentage[pattern]
    print(f"- {pattern}: {count} records ({percentage:.2f}%)")

# Visualize work patterns
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="WORK_PATTERN", data=df_schedules, order=work_pattern_counts.index, alpha=0.75)

# Add percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_schedules)
    ax.text(width + 1, p.get_y() + p.get_height()/2, f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.title("Distribution of Working Hour Patterns")
plt.xlabel("Count of Translators")
plt.ylabel("Work Pattern")
plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze schedule flexibility across weekdays
df_schedules["WEEKDAY_CONSISTENCY"] = df_schedules[weekday_cols].std(axis=1) / df_schedules[weekday_cols].mean(axis=1)
# df_schedules["WEEKDAY_CONSISTENCY"].fillna(0, inplace=True)  # Replace NaN (from division by zero) with 0

# Define flexibility categories
df_schedules["SCHEDULE_FLEXIBILITY"] = pd.cut(
    df_schedules["WEEKDAY_CONSISTENCY"],
    bins=[-0.001, 0.1, 0.3, 0.6, float('inf')], # Start at -0.001 to include 0
    labels=["Very Consistent", "Consistent", "Moderately Flexible", "Highly Flexible"]
)

# Calculate statistics
flexibility_counts = df_schedules["SCHEDULE_FLEXIBILITY"].value_counts().sort_index()
flexibility_percentage = flexibility_counts / len(df_schedules) * 100

print("Schedule Flexibility Distribution:")
for flexibility, count in flexibility_counts.items():
    percentage = flexibility_percentage[flexibility]
    print(f"  {flexibility}: {count} translators ({percentage:.2f}%)")

# Visualize schedule flexibility
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="SCHEDULE_FLEXIBILITY", data=df_schedules, 
                   order=flexibility_counts.index, alpha=0.75)
plt.title("Distribution of Schedule Flexibility Across Weekdays")
plt.xlabel("Count of Translators")
plt.ylabel("Schedule Flexibility")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_schedules)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Cross-tabulate schedule flexibility with work pattern
flexibility_vs_pattern = pd.crosstab(
    df_schedules["SCHEDULE_FLEXIBILITY"], 
    df_schedules["WORK_PATTERN"],
    normalize="index"
) * 100

In [ ]:
if "TOTAL_HOURS_PER_WEEK" not in df_schedules.columns:
    df_schedules["TOTAL_HOURS_PER_WEEK"] = df_schedules["DAILY_HOURS"] * df_schedules["ACTIVE_DAYS"]


# Identify unusual schedules for potential workload optimization

# -- Very long shifts (more than 12 hours)
long_shifts = df_schedules[df_schedules["HOURS_WORKED"] > 12]
print(f"Translators with very long shifts (>12 hours): {len(long_shifts)} ({len(long_shifts)/len(df_schedules)*100:.2f}%)")

# -- Very short shifts (less than 3 hours)
short_shifts = df_schedules[df_schedules["HOURS_WORKED"] < 3]
print(f"Translators with very short shifts (<3 hours): {len(short_shifts)} ({len(short_shifts)/len(df_schedules)*100:.2f}%)")

# -- Overnight shifts (crossing midnight)
overnight_shifts = df_schedules[df_schedules["OVERNIGHT"] == 1]
print(f"Translators with overnight shifts: {len(overnight_shifts)} ({len(overnight_shifts)/len(df_schedules)*100:.2f}%)")

# -- Weekend-only workers
weekend_only = df_schedules[
    (df_schedules[["MON", "TUES", "WED", "THURS", "FRI"]].sum(axis=1) == 0) &
    (df_schedules[["SAT", "SUN"]].sum(axis=1) > 0)
]
print(f"\nWeekend-only workers: {len(weekend_only)} ({len(weekend_only)/len(df_schedules)*100:.2f}%)\n")

# -- Very high weekly hours (more than 60 hours)
high_weekly = df_schedules[df_schedules["TOTAL_HOURS_PER_WEEK"] > 60]
print(f"Translators with high weekly hours (>60): {len(high_weekly)} ({len(high_weekly)/len(df_schedules)*100:.2f}%)")

# -- Very low weekly hours (less than 10 hours)
low_weekly = df_schedules[df_schedules["TOTAL_HOURS_PER_WEEK"] < 10]
print(f"Translators with low weekly hours (<10): {len(low_weekly)} ({len(low_weekly)/len(df_schedules)*100:.2f}%)")

# -- Single day workers
single_day = df_schedules[df_schedules["WORKING_DAYS"] == 1]
print(f"Single day workers: {len(single_day)} ({len(single_day)/len(df_schedules)*100:.2f}%)")

# -- Unusual start times (very early or late)
unusual_start = df_schedules[
    (df_schedules["START_HOUR"] < 5) | 
    ((df_schedules["START_HOUR"] > 11) & (df_schedules["START_HOUR"] < 14)) |
    (df_schedules["START_HOUR"] > 19)
]
print(f"Translators with unusual start times: {len(unusual_start)} ({len(unusual_start)/len(df_schedules)*100:.2f}%)")

all_unusual = pd.concat([ # Combine all unusual schedules and count unique translators
    long_shifts, short_shifts, overnight_shifts, weekend_only,
    high_weekly, low_weekly, single_day, unusual_start
])
unique_unusual = all_unusual["NAME"].nunique()
print(f"\nTotal unique translators with unusual schedules: {unique_unusual} ({unique_unusual/df_schedules['NAME'].nunique()*100:.2f}% of all translators)")

# Visualize the distribution of unusual schedule types
unusual_counts = {
    "Long Shifts": len(long_shifts),
    "Short Shifts": len(short_shifts),
    "Overnight Shifts": len(overnight_shifts),
    "Weekend Only": len(weekend_only),
    "High Weekly Hours": len(high_weekly),
    "Low Weekly Hours": len(low_weekly),
    "Single Day": len(single_day),
    "Unusual Start Time": len(unusual_start)
}

# Sort by frequency
unusual_counts = {k: v for k, v in sorted(unusual_counts.items(), key=lambda item: item[1], reverse=True)}

plt.figure(figsize=(14, 8))
plt.bar(unusual_counts.keys(), unusual_counts.values(), alpha=0.75)
plt.title("Distribution of Unusual Schedule Types")
plt.xlabel("Type of Unusual Schedule")
plt.ylabel("Number of Translators")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate utilization metrics
max_hours_per_week = 7 * 24

# Actual hours worked as a percentage of total possible hours
df_schedules["UTILIZATION_RATE"] = (df_schedules["TOTAL_HOURS_PER_WEEK"] / max_hours_per_week) * 100

# Calculate organizational utilization metrics
avg_utilization = df_schedules["UTILIZATION_RATE"].mean()
median_utilization = df_schedules["UTILIZATION_RATE"].median()
max_utilization = df_schedules["UTILIZATION_RATE"].max()
min_utilization = df_schedules["UTILIZATION_RATE"].min()

print("Translator Utilization Metrics:")
print(f"Average utilization rate: {avg_utilization:.2f}%")
print(f"Median utilization rate: {median_utilization:.2f}%")
print(f"Maximum utilization rate: {max_utilization:.2f}%")
print(f"Minimum utilization rate: {min_utilization:.2f}%")

# Define utilization categories
df_schedules["UTILIZATION_CATEGORY"] = pd.cut(
    df_schedules["UTILIZATION_RATE"],
    bins=[0, 10, 20, 30, 40, 50, 100],
    labels=["Very Low (<10%)", "Low (10-20%)", "Moderate (20-30%)", 
            "High (30-40%)", "Very High (40-50%)", "Extreme (>50%)"]
)

# Display utilization category distribution
util_category_counts = df_schedules["UTILIZATION_CATEGORY"].value_counts().sort_index()
util_category_percentage = util_category_counts / len(df_schedules) * 100

print("\nUtilization Category Distribution:")
for category, count in util_category_counts.items():
    percentage = util_category_percentage[category]
    print(f"  {category}: {count} translators ({percentage:.2f}%)")

# Visualize utilization categories
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="UTILIZATION_CATEGORY", data=df_schedules, 
                   order=util_category_counts.index, alpha=0.75)
plt.title("Distribution of Translator Utilization Categories")
plt.xlabel("Count of Translators")
plt.ylabel("Utilization Category")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_schedules)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate aggregate utilization metrics
total_available_translator_hours = len(df_schedules) * max_hours_per_week
total_scheduled_hours = df_schedules["TOTAL_HOURS_PER_WEEK"].sum()
organization_utilization = (total_scheduled_hours / total_available_translator_hours) * 100

print(f"\nOrganization-wide Utilization:")
print(f"Total available translator hours per week: {total_available_translator_hours:,.0f} hours")
print(f"Total scheduled hours per week: {total_scheduled_hours:,.0f} hours")
print(f"Organization-wide utilization rate: {organization_utilization:.2f}%")

In [ ]:
# Define timezone categories based on shift start times
conditions = [
    (df_schedules["START_HOUR"] >= 22) | (df_schedules["START_HOUR"] < 6),  # Late night/early morning - Asia/Pacific
    (df_schedules["START_HOUR"] >= 6) & (df_schedules["START_HOUR"] < 14),   # Morning to afternoon - Americas
    (df_schedules["START_HOUR"] >= 14) & (df_schedules["START_HOUR"] < 22)   # Evening - Europe/Africa
]
timezone_values = ["Asia/Pacific", "Americas", "Europe/Africa"]
df_schedules["LIKELY_TIMEZONE"] = np.select(conditions, timezone_values, default="Unknown")

# Calculate timezone distribution
timezone_counts = df_schedules["LIKELY_TIMEZONE"].value_counts()
timezone_percentage = timezone_counts / len(df_schedules) * 100

print("Likely Timezone Coverage:")
for timezone, count in timezone_counts.items():
    percentage = timezone_percentage[timezone]
    print(f"  {timezone}: {count} translators ({percentage:.2f}%)")

# Visualize timezone distribution
plt.figure(figsize=(14, 8))
ax = sns.barplot(x=timezone_counts.index, y=timezone_counts.values, alpha=0.75)
plt.title("Distribution of Translators by Likely Timezone Region")
plt.xlabel("Timezone Region")
plt.ylabel("Number of Translators")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    height = p.get_height()
    percentage = 100 * height / len(df_schedules)
    ax.text(p.get_x() + p.get_width()/2, height + 5, 
            f"{height} ({percentage:.1f}%)", ha="center", va="bottom")

plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Analyze 24/7 coverage
coverage_by_hour = [availability_matrix[hour, :].sum() > 0 for hour in range(24)]
all_hours_covered = all(coverage_by_hour)

print(f"\n24/7 Coverage Analysis:")
if all_hours_covered:
    print("All 24 hours have at least one translator available on at least one day of the week.")
else:
    uncovered_hours = [hour for hour, covered in enumerate(coverage_by_hour) if not covered]
    print(f"The following hours have no translator coverage on any day: {uncovered_hours}")

# Check if all days have coverage for all hours
full_day_coverage = []
for day_idx, day in enumerate(allday_cols):
    day_coverage = [availability_matrix[hour, day_idx] > 0 for hour in range(24)]
    if all(day_coverage):
        full_day_coverage.append(day)

if full_day_coverage:
    print(f"The following days have 24-hour coverage: {', '.join(full_day_coverage)}")
else:
    print("No day has complete 24-hour coverage.")

In [ ]:
print("KEY FINDINGS FROM SCHEDULE ANALYSIS:")
print()
print("Workforce Size and Structure:")
print(f"    {df_schedules['NAME'].nunique()} unique translators with varying schedule patterns")
print()
print("Working Hours:")
print(f"    Average: {df_schedules['TOTAL_HOURS_PER_WEEK'].mean():.2f} hours per week")
print(f"    Median: {df_schedules['TOTAL_HOURS_PER_WEEK'].median():.2f} hours per week")
print()
print("Shift Duration:")
print(f"    Average daily shift: {df_schedules['HOURS_WORKED'].mean():.2f} hours")
print(f"    Standard shifts (6-8h): {len(df_schedules[df_schedules['WORK_PATTERN'] == 'Standard (6-8h)'])} translators ({len(df_schedules[df_schedules['WORK_PATTERN'] == 'Standard (6-8h)'])/len(df_schedules)*100:.2f}%)")
print()
print("Schedule Patterns:")
print(f"    Distinct schedule clusters: {optimal_k}")
print(f"    Largest cluster: {cluster_counts.max()} translators ({cluster_percentages.max():.2f}% of workforce)")
print()
print("Workforce Flexibility:")
print(f"    Highly flexible schedules: {len(df_schedules[df_schedules['SCHEDULE_FLEXIBILITY'] == 'Highly Flexible'])} translators ({len(df_schedules[df_schedules['SCHEDULE_FLEXIBILITY'] == 'Highly Flexible'])/len(df_schedules)*100:.2f}%)")
print()
print("Weekend Coverage:")
print(f"    Weekend workers: {len(df_schedules[df_schedules['WEEKEND_WORKER'] == 1])} translators ({len(df_schedules[df_schedules['WEEKEND_WORKER'] == 1])/len(df_schedules)*100:.2f}%)")
print(f"    Saturday availability: {day_percentage['SAT']:.2f}%")
print(f"    Sunday availability: {day_percentage['SUN']:.2f}%")
print()
print("Timezone Coverage:")
print(f"    Americas: {timezone_percentage['Americas']:.2f}%")
print(f"    Europe/Africa: {timezone_percentage['Europe/Africa']:.2f}%")
print(f"    Asia/Pacific: {timezone_percentage['Asia/Pacific']:.2f}%")
print()
print("Peak Availability:")
print(f"    Highest availability: {allday_cols[peak_day_idx]} at {peak_hour}:00 ({hour_availability[peak_hour]:.2f}% of translators)")
print()
print("Utilization Rate:")
print(f"    Average translator utilization: {avg_utilization:.2f}%")
print(f"    Organization-wide utilization: {organization_utilization:.2f}%")
print()
print("Unusual Schedules:")
print(f"    Translators with unusual schedules: {unique_unusual} ({unique_unusual/df_schedules['NAME'].nunique()*100:.2f}%)")
print(f"    Overnight shifts: {len(overnight_shifts)} translators")
print()
print("Schedule Consistency:")
print(f"    Very consistent weekday schedules: {len(df_schedules[df_schedules['SCHEDULE_FLEXIBILITY'] == 'Very Consistent'])} translators ({len(df_schedules[df_schedules['SCHEDULE_FLEXIBILITY'] == 'Very Consistent'])/len(df_schedules)*100:.2f}%)")

#### CSV Translators Cost Pairs

In [ ]:
print("Basic information:")
df_translators.info()

In [ ]:
print(f"Number of unique translators: {df_translators['TRANSLATOR'].nunique()}")
print(f"Number of source languages: {df_translators['SOURCE_LANG'].nunique()}")
print(f"Number of target languages: {df_translators['TARGET_LANG'].nunique()}")

In [ ]:
print(f"Number of unique language pairs: {df_translators.groupby(['SOURCE_LANG', 'TARGET_LANG']).ngroups}")

In [ ]:
# Distribution of hourly rates
plt.figure(figsize=(14, 8))
plt.hist(df_translators["HOURLY_RATE"], bins=50, alpha=0.75, edgecolor="black")
plt.title("Distribution of Hourly Rates")
plt.xlabel("Hourly Rate")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Obtain the language pairs
pair_languages = df_translators.groupby(["SOURCE_LANG", "TARGET_LANG"]).size().reset_index(name="COUNT")

# Order by the most common language pairs
pair_languages_top = pair_languages.sort_values("COUNT", ascending=False).head(20)

# Order by the less common language pairs
pair_languages_bottom = pair_languages.sort_values("COUNT", ascending=True).head(20)


print("TOP Translator Pair Languages")
display(pair_languages_top)

plt.figure(figsize=(12, 10))
plt.barh(pair_languages_top.apply(lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']}", axis=1), pair_languages_top["COUNT"], alpha=0.75)
plt.xlabel("Count Frequency")
plt.ylabel("Language Pair")
plt.title("Top Common Language Pairs")
plt.show()

print("LEAST Common Translator Pair Languages")
display(pair_languages_bottom)

plt.figure(figsize=(12, 10))
plt.barh(pair_languages_bottom.apply(lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']}", axis=1), pair_languages_bottom["COUNT"], alpha=0.75)
plt.xlabel("Count Frequency")
plt.ylabel("Language Pair")
plt.xticks(range(0, 11))
plt.title("Top Uncommon Language Pairs")
plt.show()

In [ ]:
# Group data by translator and count language pairs
translator_lang_pair_counts = df_translators.groupby("TRANSLATOR").size().reset_index(name="language_pair_count")

# Print top translators
print("Translators with most language pairs:")
print(translator_lang_pair_counts.sort_values("language_pair_count", ascending=False).head(10))

# Create the plot with Seaborn
plt.figure(figsize=(12,6))
sns.histplot(
    data=translator_lang_pair_counts,
    x="language_pair_count",
    bins=25,
    stat="count",          
    alpha=0.75,
    kde=True,              
)

# Customize the plot
plt.title("Number of Language Pairs per Translator")
plt.xlabel("Number of Language Pairs")
plt.ylabel("Count of Translators")
plt.tight_layout()
plt.show()

In [ ]:
# Correlation between rates and languages
avg_rate_by_source = df_translators.groupby("SOURCE_LANG")["HOURLY_RATE"].mean().sort_values(ascending=False)
avg_rate_by_target = df_translators.groupby("TARGET_LANG")["HOURLY_RATE"].mean().sort_values(ascending=False)

print("Average hourly rate by source language:")
print(avg_rate_by_source.head(10))
print()

print("Average hourly rate by target language:")
print(avg_rate_by_target.head(10))

# Visualize average rates for top source languages
plt.figure(figsize=(14, 8))
avg_rate_by_source.head(10).plot(kind="bar", alpha=0.75)
plt.title("Average Hourly Rate by Source Language")
plt.xlabel("Source Language")
plt.ylabel("Average Hourly Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualize average rates for top target languages
plt.figure(figsize=(14, 8))
avg_rate_by_target.head(10).plot(kind="bar", alpha=0.75)
plt.title("Average Hourly Rate by Target Language")
plt.xlabel("Target Language")
plt.ylabel("Average Hourly Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed analysis of hourly rates
print("Detailed Hourly Rate Statistics:")
rate_stats = df_translators["HOURLY_RATE"].describe()
rate_stats["skew"] = df_translators["HOURLY_RATE"].skew()
rate_stats["kurtosis"] = df_translators["HOURLY_RATE"].kurtosis()
print(rate_stats)

# Define rate bins and labels based on data distribution
print(f"\nHourly Rate Range: Min = {df_translators['HOURLY_RATE'].min()}, Max = {df_translators['HOURLY_RATE'].max()}")
rate_bins = [0, 10, 15, 20, 30, 40, float('inf')]  # Adjusted to cover all possible rates
rate_labels = [
    "Very Low (≤10)",
    "Low (10-15)",
    "Medium-Low (15-20)",
    "Medium (20-30)",
    "High (30-40)",
    "Premium (>40)"
]

# Create rate categories
df_translators["RATE_CATEGORY"] = pd.cut(
    df_translators["HOURLY_RATE"],
    bins=rate_bins,
    labels=rate_labels,
    include_lowest=True
)

# Distribution of rate categories
rate_cat_counts = df_translators["RATE_CATEGORY"].value_counts().sort_index()
rate_cat_percent = rate_cat_counts / len(df_translators) * 100

print("\nHourly Rate Category Distribution:")
for category, count in rate_cat_counts.items():
    percentage = rate_cat_percent[category]
    print(f"  {category}: {count} translators ({percentage:.2f}%)")

# Visualize rate categories
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="RATE_CATEGORY", data=df_translators, 
                   order=rate_labels, alpha=0.75)  # Order by rate_labels for logical sequence
plt.title("Distribution of Hourly Rate Categories")
plt.xlabel("Count")
plt.ylabel("Rate Category")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(df_translators)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width:.0f} ({percentage:.1f}%)", ha="left", va="center")

plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze average hourly rates for translators by language pair
avg_rates_by_pair = df_translators.groupby(["SOURCE_LANG", "TARGET_LANG"])["HOURLY_RATE"].agg([
    "mean", "median", "std", "min", "max", "count"
]).reset_index()

# Sort by mean hourly rate (descending) and then by count (descending)
avg_rates_by_pair = avg_rates_by_pair.sort_values(["mean", "count"], ascending=[False, False])

# Display top highest-paid language pairs by average hourly rate
print("Top Highest-Paid Language Pairs by Average Hourly Rate for Translators:")
display(avg_rates_by_pair.head(20))

# Display top lowest-paid language pairs by average hourly rate (with at least 5 translators)
print("\nTop Lowest-Paid Language Pairs by Average Hourly Rate for Translators (min 5 translators):")
min_translators = 5
display(avg_rates_by_pair[avg_rates_by_pair["count"] >= min_translators].tail(20))

# Visualize top highest-paid language pairs by average hourly rate
top_pairs = avg_rates_by_pair.head(15)
plt.figure(figsize=(16, 10))
ax = sns.barplot(
    x="mean", 
    y=top_pairs.apply(lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']} (n={x['count']})", axis=1),
    data=top_pairs,
    alpha=0.75
)
plt.title("Top Highest-Paid Language Pairs by Average Hourly Rate for Translators")
plt.xlabel("Average Hourly Rate (€)")
plt.ylabel("Language Pair (with translator count)")

# Add rate labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    ax.text(width + 0.15, p.get_y() + p.get_height()/2, 
            f"{width:.2f}€", ha="left", va="center")

plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze rate variance within language pairs
avg_rates_by_pair["cv"] = avg_rates_by_pair["std"] / avg_rates_by_pair["mean"] # Calculate coefficient of variation (CV) for each language pair

# Filter for language pairs with at least 5 translators
min_translators = 5
rate_variance = avg_rates_by_pair[avg_rates_by_pair["count"] >= min_translators].copy()

high_variance_pairs = rate_variance.sort_values("cv", ascending=False).head(15) # Sort by coefficient of variation (descending)
low_variance_pairs = rate_variance.sort_values("cv", ascending=True).head(15) # Sort by coefficient of variation (ascending)

# Display pairs with highest rate variance
print(f"Top Language Pairs with Highest Rate Variance (min {min_translators} translators):")
display(high_variance_pairs[["SOURCE_LANG", "TARGET_LANG", "mean", "std", "min", "max", "cv", "count"]])

# Display pairs with lowest rate variance
print(f"\nTop Language Pairs with Lowest Rate Variance (min {min_translators} translators):")
display(low_variance_pairs[["SOURCE_LANG", "TARGET_LANG", "mean", "std", "min", "max", "cv", "count"]])

# Visualize rate range for high variance pairs
plt.figure(figsize=(16, 10))
high_variance_pairs_sorted = high_variance_pairs.sort_values("mean", ascending=False) # Create a custom sorted index
pair_labels = high_variance_pairs_sorted.apply(
    lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']} (CV={x['cv']:.2f})", axis=1
)
# Create error bars showing rate range
plt.errorbar(
    x=high_variance_pairs_sorted["mean"],
    y=range(len(high_variance_pairs_sorted)),
    xerr=high_variance_pairs_sorted["std"],
    fmt="o",
    capsize=5,
    alpha=0.7,
    label="Mean ± Std Dev"
)

# Add min-max range
for i, (_, row) in enumerate(high_variance_pairs_sorted.iterrows()):
    plt.plot([row["min"], row["max"]], [i, i], "k-", alpha=0.5, label="Min-Max Range" if i == 0 else "")

plt.yticks(range(len(pair_labels)), pair_labels)
plt.title("Language Pairs with Highest Rate Variance")
plt.xlabel("Hourly Rate")
plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

# Visualize rate range for low variance pairs
plt.figure(figsize=(16, 10))
low_variance_pairs_sorted = low_variance_pairs.sort_values("mean", ascending=False) # Create a custom sorted index
pair_labels = low_variance_pairs_sorted.apply(
    lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']} (CV={x['cv']:.2f})", axis=1
)
# Create error bars showing rate range
plt.errorbar(
    x=low_variance_pairs_sorted["mean"],
    y=range(len(low_variance_pairs_sorted)),
    xerr=low_variance_pairs_sorted["std"],
    fmt="o",
    capsize=5,
    alpha=0.7,
    label="Mean ± Std Dev"
)
# Add min-max range
for i, (_, row) in enumerate(low_variance_pairs_sorted.iterrows()):
    plt.plot([row["min"], row["max"]], [i, i], "k-", alpha=0.5, label="Min-Max Range" if i == 0 else "")
plt.yticks(range(len(pair_labels)), pair_labels)
plt.title("Language Pairs with Lowest Rate Variance")
plt.xlabel("Hourly Rate")
plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Analyze translators by number of language pairs

pair_count_quantiles = translator_lang_pair_counts["language_pair_count"].quantile([0, 0.25, 0.5, 0.75, 0.9, 1]) # Calculate quantiles for language pair counts
print("Language Pair Count Quantiles:")
print(pair_count_quantiles)

# Create translator specialization categories
trans_specialization_bins = [0, 1, 3, 6, 10, 100]  # Adjust based on your distribution
trans_specialization_labels = [
    "Single Specialty (1)", 
    "Limited Range (2-3)", 
    "Moderate Range (4-6)", 
    "Wide Range (7-10)", 
    "Extensive Range (>10)"
]

translator_lang_pair_counts["SPECIALIZATION"] = pd.cut(
    translator_lang_pair_counts["language_pair_count"],
    bins=trans_specialization_bins,
    labels=trans_specialization_labels,
    include_lowest=True
)

# Distribution of translator specialization
specialization_counts = translator_lang_pair_counts["SPECIALIZATION"].value_counts().sort_index()
specialization_percent = specialization_counts / len(translator_lang_pair_counts) * 100

print("\nTranslator Specialization Distribution:")
for category, count in specialization_counts.items():
    percentage = specialization_percent[category]
    print(f"  {category}: {count} translators ({percentage:.2f}%)")

# Visualize translator specialization
plt.figure(figsize=(14, 8))
ax = sns.countplot(y="SPECIALIZATION", data=translator_lang_pair_counts, 
                   order=specialization_counts.index, alpha=0.75)
plt.title("Distribution of Translator Specialization")
plt.xlabel("Count of Translators")
plt.ylabel("Specialization Category")

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / len(translator_lang_pair_counts)
    ax.text(width + 5, p.get_y() + p.get_height()/2, 
            f"{width} ({percentage:.1f}%)", ha="left", va="center")

plt.grid(True, axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze relationship between specialization and hourly rates
translator_avg_rates = df_translators.groupby("TRANSLATOR")["HOURLY_RATE"].mean().reset_index() # Calculate average hourly rate per translator
translator_avg_rates.rename(columns={"HOURLY_RATE": "AVG_HOURLY_RATE"}, inplace=True)

translator_analysis = pd.merge( # Merge with translator language pair counts
    translator_lang_pair_counts,
    translator_avg_rates,
    on="TRANSLATOR"
)

if "SPECIALIZATION" not in translator_analysis.columns:
    # Define specialization categories based on language pair count
    trans_specialization_bins = [0, 1, 3, 6, 10, 70]  # Boundaries for specialization categories
    trans_specialization_labels = [
        "Single Specialty (1)", 
        "Limited Range (2-3)", 
        "Moderate Range (4-6)", 
        "Wide Range (7-10)", 
        "Extensive Range (>10)"
    ]
    
    # Create specialization categories
    translator_analysis["SPECIALIZATION"] = pd.cut(
        translator_analysis["language_pair_count"],
        bins=trans_specialization_bins,
        labels=trans_specialization_labels,
        include_lowest=True
    )

# Calculate average rate by specialization category with observed=True to silence warning
specialization_rates = translator_analysis.groupby("SPECIALIZATION", observed=True).agg({
    "AVG_HOURLY_RATE": ["mean", "median", "std", "min", "max", "count"]
}).round(2)

print("Average Hourly Rates by Specialization:")
display(specialization_rates)

# Get specialization counts for proper ordering in plots
specialization_counts = translator_analysis["SPECIALIZATION"].value_counts().sort_index()

# Visualize relationship between specialization and rates
plt.figure(figsize=(14, 8))
sns.boxplot(
    x="SPECIALIZATION", 
    y="AVG_HOURLY_RATE", 
    data=translator_analysis.dropna(subset=["SPECIALIZATION"]),  # Drop NaNs to avoid empty boxes
    order=specialization_counts.index,

)
plt.title("Relationship Between Translator Specialization and Hourly Rates")
plt.xlabel("Specialization Category")
plt.ylabel("Average Hourly Rate")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Create a scatter plot of pair count vs hourly rate
plt.figure(figsize=(14, 8))
sns.scatterplot(
    x="language_pair_count", 
    y="AVG_HOURLY_RATE",
    data=translator_analysis, 
    alpha=0.6
)
plt.title("Relationship Between Number of Language Pairs and Average Hourly Rate")
plt.xlabel("Number of Language Pairs")
plt.ylabel("Average Hourly Rate")
plt.grid(True, linestyle="--", alpha=0.7)

# Add a trend line
sns.regplot(
    x="language_pair_count", 
    y="AVG_HOURLY_RATE",
    data=translator_analysis, 
    scatter=False,  # Don't add additional scatter points
    line_kws={"color": "red"}
)
plt.ylabel("Average Hourly Rate")
plt.xlabel("Number of Language Pairs")
plt.tight_layout()
plt.show()

# Calculate correlation
corr = translator_analysis[["language_pair_count", "AVG_HOURLY_RATE"]].corr().iloc[0, 1]
print(f"Correlation between number of language pairs and average hourly rate: {corr:.4f}")

# Add additional insights about the relationship
min_rate = translator_analysis.loc[translator_analysis["language_pair_count"] == 1, "AVG_HOURLY_RATE"].mean()
max_rate = translator_analysis.loc[translator_analysis["language_pair_count"] > 10, "AVG_HOURLY_RATE"].mean()
rate_increase = max_rate - min_rate
percent_increase = (rate_increase / min_rate) * 100 if min_rate > 0 else 0

print(f"\nAdditional Insights:")
print(f"- Translators with only one language pair earn on average {min_rate:.2f}€/hour")
print(f"- Translators with more than 10 language pairs earn on average {max_rate:.2f}€/hour")
print(f"- This represents a difference of {rate_increase:.2f}€/hour or {percent_increase:.1f}% higher rates for diversified translators")
print(f"- The positive correlation of {corr:.4f} suggests that specializing in more language pairs is associated with higher compensation")

In [ ]:
# Analyze language rarity and its impact on rates
source_counts = df_translators["SOURCE_LANG"].value_counts().reset_index() # Count translators for each language
source_counts.columns = ["LANGUAGE", "SOURCE_COUNT"]
target_counts = df_translators["TARGET_LANG"].value_counts().reset_index()
target_counts.columns = ["LANGUAGE", "TARGET_COUNT"]

# Merge to get total counts
language_counts = pd.merge(source_counts, target_counts, on="LANGUAGE", how="outer").fillna(0)
language_counts["TOTAL_COUNT"] = language_counts["SOURCE_COUNT"] + language_counts["TARGET_COUNT"]
language_counts.sort_values("TOTAL_COUNT", ascending=False, inplace=True)

print("Most Common Languages (Source or Target):")
display(language_counts.head(15))

print("\nRarest Languages (Source or Target):")
display(language_counts.tail(15))

# Calculate average rate for each language (when used as source or target)
source_rates = df_translators.groupby("SOURCE_LANG")["HOURLY_RATE"].mean().reset_index()
source_rates.columns = ["LANGUAGE", "SOURCE_RATE"]

target_rates = df_translators.groupby("TARGET_LANG")["HOURLY_RATE"].mean().reset_index()
target_rates.columns = ["LANGUAGE", "TARGET_RATE"]

# Merge counts with rates
language_analysis = pd.merge(language_counts, source_rates, on="LANGUAGE", how="left")
language_analysis = pd.merge(language_analysis, target_rates, on="LANGUAGE", how="left")

# Calculate weighted average rate
language_analysis["AVG_RATE"] = (
    (language_analysis["SOURCE_RATE"] * language_analysis["SOURCE_COUNT"] + 
     language_analysis["TARGET_RATE"] * language_analysis["TARGET_COUNT"]) / 
    language_analysis["TOTAL_COUNT"]
)

# Create rarity categories
rarity_bins = [0, 10, 50, 100, 200, 500, 10000]
rarity_labels = ["Very Rare (<10)", "Rare (10-50)", "Uncommon (51-100)", 
                "Common (101-200)", "Very Common (201-500)", "Abundant (>500)"]

language_analysis["RARITY"] = pd.cut(
    language_analysis["TOTAL_COUNT"],
    bins=rarity_bins,
    labels=rarity_labels,
    include_lowest=True
)

# Analyze relationship between language rarity and rates
rarity_rates = language_analysis.groupby("RARITY").agg({
    "AVG_RATE": ["mean", "median", "std", "count"]
}).round(2)

print("\nRelationship Between Language Rarity and Rates:")
display(rarity_rates)

# Visualize relationship
plt.figure(figsize=(14, 8))
rarity_means = language_analysis.groupby("RARITY")["AVG_RATE"].mean().sort_index()
ax = sns.barplot(x=rarity_means.index, y=rarity_means.values, alpha=0.75)
plt.title("Average Hourly Rate by Language Rarity")
plt.xlabel("Language Rarity")
plt.ylabel("Average Hourly Rate")

# Add rate labels
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2, height + 0.5, 
            f"{height:.2f}€", ha="center", va="bottom")

plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Scatter plot of language count vs. rate
plt.figure(figsize=(14, 8))
plt.scatter(language_analysis["TOTAL_COUNT"], language_analysis["AVG_RATE"], alpha=0.6)
plt.title("Relationship Between Language Frequency and Average Rate")
plt.xlabel("Language Frequency (Total Count)")
plt.ylabel("Average Hourly Rate")
plt.xscale("log")  # Use log scale for better visualization
plt.grid(True, linestyle="--", alpha=0.7)

# Add labels for some points
for idx, row in language_analysis.iterrows():
    if row["AVG_RATE"] > 40 or row["TOTAL_COUNT"] < 5 or row["TOTAL_COUNT"] > 500:
        plt.annotate(row["LANGUAGE"], 
                    (row["TOTAL_COUNT"], row["AVG_RATE"]),
                    xytext=(5, 5),
                    textcoords="offset points")
plt.tight_layout()
plt.show()

In [ ]:
# Analyze language direction premium (source vs. target)
direction_analysis = pd.merge(  # For languages that appear as both source and target
    source_rates, 
    target_rates, 
    on="LANGUAGE", 
    how="inner",
    suffixes=("_SOURCE", "_TARGET")
)

# Calculate premium (positive if target rate > source rate)
direction_analysis["DIRECTION_PREMIUM"] = direction_analysis["TARGET_RATE"] - direction_analysis["SOURCE_RATE"]
direction_analysis["PREMIUM_PCT"] = (direction_analysis["DIRECTION_PREMIUM"] / direction_analysis["SOURCE_RATE"]) * 100

# Merge with language counts for context
direction_analysis = pd.merge(
    direction_analysis,
    language_counts[["LANGUAGE", "TOTAL_COUNT"]],
    on="LANGUAGE"
)

# Filter to languages with reasonable frequency
min_freq = 10
direction_filtered = direction_analysis[direction_analysis["TOTAL_COUNT"] >= min_freq].copy()

# Sort by premium
source_premium = direction_filtered.sort_values("DIRECTION_PREMIUM", ascending=False)
target_premium = direction_filtered.sort_values("DIRECTION_PREMIUM", ascending=True)

print(f"Top Languages with Source > Target Premium (min freq {min_freq}):")
display(source_premium.head(15))

print(f"\nTop Languages with Target > Source Premium (min freq {min_freq}):")
display(target_premium.head(15))

# Visualize direction premium
plt.figure(figsize=(16, 10))
combined_premium = pd.concat([source_premium.head(10), target_premium.head(10)])
combined_premium = combined_premium.drop_duplicates(subset=["LANGUAGE"])
combined_premium = combined_premium.sort_values("DIRECTION_PREMIUM")

ax = sns.barplot(
    x="DIRECTION_PREMIUM", 
    y="LANGUAGE",
    data=combined_premium,
    palette="coolwarm_r",  # Red for negative, blue for positive
    alpha=0.75
)
plt.title("Hourly Rate Difference: Translating To vs. From Languages")
plt.xlabel("Hourly Rate Difference (€)")
plt.ylabel("Language")
plt.axvline(x=0, color="black", linestyle="--")
plt.grid(True, axis="x", linestyle="--", alpha=0.7)

# Add improved labels without percentage
for i, p in enumerate(ax.patches):
    width = p.get_width()
    abs_width = abs(width)
    label = f"€{abs_width:.2f} T>S" if width >= 0 else f"€{abs_width:.2f} S>T"
    label_pos = width - 2 if width >= 0 else width + 2
    ax.text(label_pos, p.get_y() + p.get_height()/2, 
            label, ha="left" if width >= 0 else "right", va="center")

plt.tight_layout()
plt.show()

In [ ]:
# Cluster analysis for language pairs

cluster_data = avg_rates_by_pair.copy() # Use language pair frequency and rate information

# Simplify data for clustering
cluster_features = [
    "mean",   # Average hourly rate
    "std",    # Standard deviation of rates
    "count"   # Number of translators
]

min_trans = 3 # Filter to pairs with at least 3 translators for meaningful analysis
cluster_data = cluster_data[cluster_data["count"] >= min_trans]

# Standardize data for KMeans
cluster_data_scaled = (cluster_data[cluster_features] - cluster_data[cluster_features].mean()) / \
                       cluster_data[cluster_features].std()

# Determine optimal number of clusters
inertia = []
k_range = range(1, 10)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(cluster_data_scaled)
    inertia.append(kmeans.inertia_)

# Plot elbow method
plt.figure(figsize=(14, 8))
plt.plot(k_range, inertia, marker="o")
plt.title("Elbow Method for Language Pair Cluster Selection")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
optimal_k = 4  # Adjust based on the elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_data["CLUSTER"] = kmeans.fit_predict(cluster_data_scaled)

# Analyze the clusters
cluster_stats = cluster_data.groupby("CLUSTER").agg({
    "mean": ["mean", "min", "max"],
    "std": ["mean"],
    "count": ["mean", "min", "max", "sum"],
    "SOURCE_LANG": "count"
}).round(2)

# Rename columns for better readability
cluster_stats.columns = [
    "Avg_Rate_Mean", "Avg_Rate_Min", "Avg_Rate_Max", 
    "Std_Dev_Mean", 
    "Translators_Mean", "Translators_Min", "Translators_Max", "Total_Translators",
    "Lang_Pair_Count"
]

print("Language Pair Cluster Statistics:")
display(cluster_stats)

# Calculate percentage by cluster
cluster_counts = cluster_data["CLUSTER"].value_counts().sort_index()
cluster_percentages = cluster_counts / len(cluster_data) * 100

print("\nCluster Distribution:")
for cluster, count in cluster_counts.items():
    percentage = cluster_percentages[cluster]
    print(f"Cluster {cluster}: {count} language pairs ({percentage:.2f}%)")

# Visualize the clusters
plt.figure(figsize=(14, 8))
scatter = plt.scatter(
    cluster_data["count"],
    cluster_data["mean"],
    c=cluster_data["CLUSTER"],
    cmap="YlGnBu",
    s=100,
    alpha=0.6
)
plt.colorbar(scatter, label="Cluster")
plt.title("Language Pair Clusters by Translator Count and Average Rate")
plt.xlabel("Number of Translators")
plt.ylabel("Average Hourly Rate")
plt.grid(True, linestyle="--", alpha=0.7)

# Add cluster centroids
centroids = kmeans.cluster_centers_
centroids_unscaled = centroids * cluster_data[cluster_features].std().values + \
                    cluster_data[cluster_features].mean().values
plt.scatter(
    centroids_unscaled[:, 2],  # count feature
    centroids_unscaled[:, 0],  # mean feature
    s=50,
    marker="X",
    c="red",
    label="Centroids"
)
plt.legend()
plt.tight_layout()
plt.show()

# Assign cluster labels based on characteristics
cluster_labels = {
    # Customize based on actual clusters
    0: "Premium Specialized",
    1: "Standard Common",
    2: "Budget Common", 
    3: "Rare Specialty"
}

# Map cluster numbers to labels
cluster_data["CLUSTER_LABEL"] = cluster_data["CLUSTER"].map(cluster_labels)

# Sample of language pairs in each cluster
print("\nSample Language Pairs by Cluster:")
for cluster, label in cluster_labels.items():
    cluster_sample = cluster_data[cluster_data["CLUSTER"] == cluster].head(5)
    print(f"\nCluster {cluster}: {label}")
    for _, row in cluster_sample.iterrows():
        print(f"  {row['SOURCE_LANG']} > {row['TARGET_LANG']}: €{row['mean']:.2f} avg rate, {row['count']} translators")

In [ ]:
# Calculate average rates for high-demand language pairs
top_pairs = pair_languages_top.copy()  # Use the top most common pairs from previous analysis

# Merge with rate information
top_pairs_with_rates = pd.merge(
    top_pairs,
    avg_rates_by_pair[["SOURCE_LANG", "TARGET_LANG", "mean", "median", "std", "min", "max", "count"]],
    on=["SOURCE_LANG", "TARGET_LANG"]
)

# Sort by count descending and show rate information
top_pairs_with_rates = top_pairs_with_rates.sort_values("COUNT", ascending=False)
print("Rate Information for Top Most In-Demand Language Pairs:")
display(top_pairs_with_rates)

# Calculate total cost for top pairs
top_pairs_with_rates["TOTAL_COST"] = top_pairs_with_rates["COUNT"] * top_pairs_with_rates["mean"]
top_pairs_with_rates["COST_PERCENTAGE"] = (top_pairs_with_rates["TOTAL_COST"] / top_pairs_with_rates["TOTAL_COST"].sum()) * 100

print("\nCost Analysis for Top Language Pairs:")
print(f"Total cost for Top pairs: {top_pairs_with_rates['TOTAL_COST'].sum():.2f}€")

# Visualize cost distribution for top pairs
plt.figure(figsize=(16, 10))
top_10_cost = top_pairs_with_rates.head(10).copy()
top_10_cost["PAIR_LABEL"] = top_10_cost.apply(
    lambda x: f"{x['SOURCE_LANG']} $\\rightarrow$ {x['TARGET_LANG']} (€{x['mean']:.2f})", axis=1
)

# Generate colors from YlGnBu colormap
cmap = cm.get_cmap("YlGnBu")
colors = [cmap(i / 10) for i in range(10)]

plt.pie(
    top_10_cost["TOTAL_COST"], 
    labels=top_10_cost["PAIR_LABEL"],
    autopct="%1.1f%%",
    startangle=90,
    shadow=False,
    explode=[0.1 if i == 0 else 0.05 if i == 1 else 0 for i in range(10)],
    colors=colors
)
plt.title("Cost Distribution Among top Language Pairs")
plt.axis("equal")  # Equal aspect ratio ensures that pie is drawn as a circle
plt.tight_layout()
plt.show()

In [ ]:
# Analyze rate competitiveness and translator distribution
df_translators["RATE_ZSCORE"] = (
    df_translators["HOURLY_RATE"] - df_translators["HOURLY_RATE"].mean()
) / df_translators["HOURLY_RATE"].std()

conditions = [ # Create rate competitiveness categories
    (df_translators["RATE_ZSCORE"] >= -2) & (df_translators["RATE_ZSCORE"] < -1),
    (df_translators["RATE_ZSCORE"] >= -1) & (df_translators["RATE_ZSCORE"] < 1),
    (df_translators["RATE_ZSCORE"] >= 1) & (df_translators["RATE_ZSCORE"] < 2),
    (df_translators["RATE_ZSCORE"] >= 2)
]

comp_values = ["Competitive", "Average", "Premium", "Very Premium"]
df_translators["COMPETITIVENESS"] = np.select(conditions, comp_values, default="Average")

# Competitiveness distribution - explicitly define the order to ensure consistency
competitiveness_order = ["Competitive", "Average", "Premium", "Very Premium"]
comp_counts = df_translators["COMPETITIVENESS"].value_counts()
# Reindex to ensure all categories are present with consistent order
comp_counts = comp_counts.reindex(competitiveness_order, fill_value=0)
comp_percent = comp_counts / len(df_translators) * 100

print("Rate Competitiveness Distribution:")
for comp in competitiveness_order:
    count = comp_counts[comp]
    percentage = comp_percent[comp]
    print(f"  {comp}: {count} translator rates ({percentage:.2f}%)")

# Visualize rate competitiveness - use the same explicit order
plt.figure(figsize=(14, 8))
sns.countplot(
    x="COMPETITIVENESS", 
    data=df_translators, 
    order=competitiveness_order,
    alpha=0.75
)
plt.title("Distribution of Rate Competitiveness")
plt.xlabel("Competitiveness Category")
plt.ylabel("Count")
plt.grid(True, axis="y", linestyle="--")
for i, p in enumerate(plt.gca().patches): # Add count labels
    width = p.get_height()
    plt.text(p.get_x() + p.get_width()/2, width + 5, 
             f"{width:.0f}", ha="center", va="bottom")
plt.tight_layout()
plt.show()

# Analyze competitiveness by language pair
comp_by_pair = df_translators.groupby(["SOURCE_LANG", "TARGET_LANG"])["COMPETITIVENESS"].value_counts(normalize=True)
comp_by_pair = comp_by_pair.mul(100).rename("PERCENTAGE").reset_index()

# Identify language pairs with most competitive rates - being explicit about the categories
competitive_categories = ["Very Competitive", "Competitive"]
top_competitive_pairs = comp_by_pair[
    (comp_by_pair["COMPETITIVENESS"].isin(competitive_categories)) &
    (comp_by_pair["PERCENTAGE"] > 50)  # At least 50% of translators have competitive rates
].sort_values("PERCENTAGE", ascending=False)

print("\nLanguage Pairs with Most Competitive Rates:")
display(top_competitive_pairs.head(10))

# Identify language pairs with most premium rates - being explicit about the categories
premium_categories = ["Premium", "Very Premium"]
top_premium_pairs = comp_by_pair[
    (comp_by_pair["COMPETITIVENESS"].isin(premium_categories)) &
    (comp_by_pair["PERCENTAGE"] > 50)  # At least 50% of translators have premium rates
].sort_values("PERCENTAGE", ascending=False)

print("\nLanguage Pairs with Most Premium Rates:")
display(top_premium_pairs.head(10))

# Calculate the total counts and percentages for key categories
competitive_count = comp_counts["Competitive"]
competitive_percent = comp_percent["Competitive"]
premium_count = comp_counts["Premium"] + comp_counts["Very Premium"]
premium_percent = comp_percent["Premium"] + comp_percent["Very Premium"]

# Print a summary for easy reference
print("\nSummary statistics:")
print(f"Competitive rates: {competitive_count} ({competitive_percent:.2f}%)")
print(f"Premium rates: {premium_count} ({premium_percent:.2f}%)")

In [ ]:
print("KEY FINDINGS FROM TRANSLATOR COST PAIRS ANALYSIS:")
print()
print("Translator Pool Size:")
print(f"    {df_translators['TRANSLATOR'].nunique()} unique translators across {df_translators.groupby(['SOURCE_LANG', 'TARGET_LANG']).ngroups} distinct language pairs")
print()
print("Language Coverage:")
print(f"    Source languages: {df_translators['SOURCE_LANG'].nunique()}")
print(f"    Target languages: {df_translators['TARGET_LANG'].nunique()}")
print()
print("Rate Distribution:")
print(f"    Average hourly rate: {df_translators['HOURLY_RATE'].mean():.2f}€")
print(f"    Rate range: {df_translators['HOURLY_RATE'].min():.2f}€ to {df_translators['HOURLY_RATE'].max():.2f}€")
print(f"    Median rate: {df_translators['HOURLY_RATE'].median():.2f}€")
print()
print("Specialization Patterns:")
print(f"    Single language pair specialists: {len(translator_lang_pair_counts[translator_lang_pair_counts['language_pair_count'] == 1])} translators ({len(translator_lang_pair_counts[translator_lang_pair_counts['language_pair_count'] == 1])/len(translator_lang_pair_counts)*100:.2f}%)")
print(f"    Highly versatile translators (>10 pairs): {len(translator_lang_pair_counts[translator_lang_pair_counts['language_pair_count'] > 10])} translators ({len(translator_lang_pair_counts[translator_lang_pair_counts['language_pair_count'] > 10])/len(translator_lang_pair_counts)*100:.2f}%)")
print()
print("Rate Variation:")
print(f"    Correlation between language pairs and rate: {corr:.4f}")
print()
print("Language Rarity Premium:")
print(f"    Very rare languages: {language_analysis[language_analysis['RARITY'] == 'Very Rare (<10)']['AVG_RATE'].mean():.2f}€")
print(f"    Abundant languages: {language_analysis[language_analysis['RARITY'] == 'Abundant (>500)']['AVG_RATE'].mean():.2f}€")
print()
print("Popular Language Pairs:")
print(f"    Most common: {pair_languages_top.iloc[0]['SOURCE_LANG']} → {pair_languages_top.iloc[0]['TARGET_LANG']} with {pair_languages_top.iloc[0]['COUNT']} translators")
print(f"    Second most common: {pair_languages_top.iloc[1]['SOURCE_LANG']} → {pair_languages_top.iloc[1]['TARGET_LANG']} with {pair_languages_top.iloc[1]['COUNT']} translators")
print()
print("Rare Language Pairs:")
print(f"    Single-translator pairs: {len(pair_languages[pair_languages['COUNT'] == 1])} ({len(pair_languages[pair_languages['COUNT'] == 1])/len(pair_languages)*100:.2f}% of all pairs)")
print()
print("Direction Premium:")
print(f"    Highest premium: {direction_analysis.iloc[direction_analysis['DIRECTION_PREMIUM'].argmax()]['LANGUAGE']} at {direction_analysis.iloc[direction_analysis['DIRECTION_PREMIUM'].argmax()]['DIRECTION_PREMIUM']:.2f}€ when used as target language")
print()
print("Rate Clusters:")
print(f"    Distinct rate clusters: {optimal_k}")
print(f"    Largest cluster: {cluster_counts.max()} pairs ({cluster_percentages.max():.2f}% of all pairs)")
print()
print("Rate Competitiveness:")
print(f"    Competitive rates: {comp_counts['Competitive']} ({(comp_percent['Competitive']):.2f}%)")
print(f"    Premium/very premium rates: {comp_counts['Premium'] + comp_counts['Very Premium']} ({(comp_percent['Premium'] + comp_percent['Very Premium']):.2f}%)")
print()
print("High-Value Language Pairs:")
print(f"    Top pairs by volume: {top_10_cost['COST_PERCENTAGE'].sum():.2f}% of total translation cost")
print(f"    Highest volume pair: {top_10_cost.iloc[0]['SOURCE_LANG']} → {top_10_cost.iloc[0]['TARGET_LANG']} ({top_10_cost.iloc[0]['COST_PERCENTAGE']:.2f}% of cost)")

#### CSV Sample

In [ ]:
print("Basic information:")
df_data.info()

In [ ]:
print("Missing values by column:")
print(df_data.isnull().sum())

In [ ]:
# Convert timestamp columns to datetime
timestamp_cols = ["START", "END", "ASSIGNED", "READY", "WORKING", "DELIVERED", "RECEIVED", "CLOSE"]
for col in timestamp_cols:
    if col in df_data.columns:
        df_data[col] = pd.to_datetime(df_data[col], errors="coerce")

print("Data types after conversion:")
print(df_data.dtypes)

In [ ]:
# Display the first few rows
print("Sample data:")
display(df_data.head())

In [ ]:
# Display basic information
print(f"Number of rows: {df_data.shape[0]}")
print(f"Number of columns: {df_data.shape[1]}")
print(f"Number of unique projects: {df_data['PROJECT_ID'].nunique()}")
print(f"Number of unique tasks: {df_data['TASK_ID'].nunique()}")
print(f"Number of unique translators: {df_data['TRANSLATOR'].nunique()}")
print(f"Number of unique project managers: {df_data['PM'].nunique()}")


In [ ]:
# DONE
# Duplicate values of TASK_ID exists
# - This might be an issue with the data collection
# - Or simply a different project with the same task id
# - Time information indicates that these are different tasks but with the same id

df_data[df_data["TASK_ID"].duplicated()]

In [ ]:
# Count unique values for categorical columns
categorical_cols = ["PM", "TASK_TYPE", "SOURCE_LANG", "TARGET_LANG", 
                   "TRANSLATOR", "MANUFACTURER", "MANUFACTURER_SECTOR",
                   "MANUFACTURER_INDUSTRY_GROUP", "MANUFACTURER_INDUSTRY", 
                   "MANUFACTURER_SUBINDUSTRY"]

print("Unique values for categorical columns:")
for col in categorical_cols:
    print(f"{col}: {df_data[col].nunique()} unique values")

In [ ]:
# Columns with repetitive information
df_data[["MANUFACTURER", "MANUFACTURER_SECTOR", "MANUFACTURER_INDUSTRY_GROUP", "MANUFACTURER_INDUSTRY", "MANUFACTURER_SUBINDUSTRY"]].head()

In [ ]:
# Basic information
print(f"Number of unique projects: {df_data['PROJECT_ID'].nunique()}")
print(f"Number of unique tasks: {df_data['TASK_ID'].nunique()}")
print(f"Number of unique translators: {df_data['TRANSLATOR'].nunique()}")
print(f"Number of source languages: {df_data['SOURCE_LANG'].nunique()}")
print(f"Number of target languages: {df_data['TARGET_LANG'].nunique()}")
print(f"Number of unique task types: {df_data['TASK_TYPE'].nunique()}")

In [ ]:
# Summary statistics for numeric columns
print("Summary statistics for numeric columns:")
numeric_cols = df_data.select_dtypes(include=["int64", "float64"]).columns
display(df_data[numeric_cols].describe())

In [ ]:
# Check distribution of quality evaluations (quality control scores)
plt.figure(figsize=(14, 8))
sns.histplot(df_data["QUALITY_EVALUATION"].dropna(), bins=10, alpha=0.75)
plt.title("Distribution of Quality Evaluations")
plt.xlabel("Quality Score")
plt.ylabel("Count of Tasks")
plt.show()

In [ ]:
# Translator hourly compensation rates
plt.figure(figsize=(14, 8))
sns.histplot(df_data["HOURLY_RATE"].dropna(), kde=True, alpha=0.75)
plt.title("Translator Hourly Compensation Rates")
plt.xlabel("Hourly Rate")
plt.ylabel("Number of Translators")
plt.tight_layout()
plt.show()

# Total project costs
plt.figure(figsize=(14, 8))
sns.histplot(df_data["COST"].dropna(), kde=False, alpha=0.75)
plt.title("Distribution of Project Costs")
plt.xlim(0, 150)
plt.xlabel("Cost")
plt.ylabel("Number of Projects")
plt.tight_layout()
plt.show()

In [ ]:
# Indicate the top projects with higher task counts
top_projects = df_data["PROJECT_ID"].value_counts().head(10)
print("Top projects with higher task counts:")
for project, count in top_projects.items():
    print(f"\t{project}: {count} tasks")

# Indicate the lowest projects with lower task counts
bottom_projects = df_data["PROJECT_ID"].value_counts().tail(10)
print("\nLowest projects with lower task counts:")
for project, count in bottom_projects.items():
    print(f"\t{project}: {count} tasks")


# Analyze tasks per project
tasks_per_project = df_data.groupby("PROJECT_ID").size()
print("\nTasks per Project Statistics:")
print(tasks_per_project.describe())

plt.figure(figsize=(14, 8))
sns.histplot(tasks_per_project, bins=1000, kde=True, edgecolor="black", alpha=0.75)
plt.title("Distribution of Tasks per Project")
plt.xlabel("Number of Tasks")
plt.ylabel("Count of Projects")
plt.xlim(0, 100)
plt.tight_layout()
plt.show()

In [ ]:
# IMPORTANT

# Ranges of taks per project to get an idea of the distribution
bins = [0, 2, 5, 10, 20, 35, 50, 100, 150, 200, 500, 1000, 2000, 5000, 10000, float("inf")]
labels = ["0-2", "2-5", "5-10", "10-20", "20-35", "35-50", "50-100", "100-150", "150-200", "200-500", "500-1000", "1000-2000", "2000-5000", "5000-10000", ">10000"]

# Bin tasks_per_project
task_ranges = pd.cut(
    tasks_per_project,
    bins=bins,
    labels=labels,
    right=False
)

task_range_counts = task_ranges.value_counts().sort_index() # Count projects in each range
task_range_percentages = task_range_counts / len(tasks_per_project) * 100 # Calculate percentages

# Display the results
print("Task Range Distribution:")
for range_label, count in task_range_counts.items():
    percentage = task_range_percentages[range_label]
    print(f"  {range_label}: {count} projects ({percentage:.2f}%)")


plt.figure(figsize=(14, 8))
plt.bar(task_range_counts.index, task_range_counts.values, alpha=0.75)
plt.title("Distribution of Tasks per Project Ranges")
plt.xlabel("Task Range")
plt.ylabel("Number of Projects")
plt.xticks(rotation=45)
# Add count labels
for i, count in enumerate(task_range_counts):
    plt.text(i, count + 5, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Analyze project distribution
projects_per_pm = df_data.groupby("PM")["PROJECT_ID"].nunique()
tasks_per_pm = df_data.groupby("PM").size()
# print(tasks_per_pm)

pm_stats = pd.DataFrame({
    "Projects": projects_per_pm,
    "Tasks": tasks_per_pm,
    "Avg Tasks/Project": tasks_per_pm / projects_per_pm
}).sort_values("Tasks", ascending=True)

print("Project Manager Workload:")
display(pm_stats)

# Visualize PM workload
plt.figure(figsize=(14, 8))
sns.barplot(x=pm_stats.head(15).index, y=pm_stats.head(15)["Tasks"], alpha=0.75)
plt.title("Top Project Managers by Number of Tasks")
plt.xlabel("Project Manager")
plt.ylabel("Number of Tasks")
# Add count labels
for i, count in enumerate(pm_stats.head(15)["Tasks"]):
    plt.text(i, count + 5, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Task Type Analysis
task_type_counts = df_data["TASK_TYPE"].value_counts()
task_type_percentage = task_type_counts / len(df_data) * 100

print("Task Type Distribution:")
for task_type, count in task_type_counts.items():
    percentage = task_type_percentage[task_type]
    print(f"  {task_type}: {count} tasks ({percentage:.2f}%)")

plt.figure(figsize=(14, 8))
sns.countplot(y="TASK_TYPE", data=df_data, order=task_type_counts.index, alpha=0.75)
plt.ylabel("Translation Service Category")
plt.xlabel("Number of Tasks")
plt.title("Distribution of Translation Service Categories")
plt.tight_layout()
plt.show()

In [ ]:
for col in ["ASSIGNED", "DELIVERED"]: # Ensure date columns are in datetime format
    if not pd.api.types.is_datetime64_dtype(df_data[col]):
        df_data[col] = pd.to_datetime(df_data[col], errors="coerce")

# Calculate raw timedelta (difference between DELIVERED and ASSIGNED)
mask = (~df_data["ASSIGNED"].isnull()) & (~df_data["DELIVERED"].isnull())
df_data.loc[mask, "TURNAROUND_TIME"] = df_data.loc[mask, "DELIVERED"] - df_data.loc[mask, "ASSIGNED"]
df_data_filtered = df_data.copy() # Keep changes in a new DataFrame

# Filter out unreasonable values (Negative turnaround times)
reasonable_mask = df_data_filtered["TURNAROUND_TIME"].dt.total_seconds() > 0
df_data_filtered = df_data_filtered.loc[reasonable_mask].sort_values("TURNAROUND_TIME") # DataFrame sorted by turnaround time


# Display shortest and longest tasks
print("Shortest 5 tasks by turnaround time:")
display(df_data_filtered[["ASSIGNED", "DELIVERED", "TURNAROUND_TIME"]].head(5))
print("Longest 5 tasks by turnaround time:")
display(df_data_filtered[["ASSIGNED", "DELIVERED", "TURNAROUND_TIME"]].tail(5))

# Visualize distribution of turnaround times (Transformed to hours)
plt.figure(figsize=(14, 8))
plt.hist(df_data_filtered["TURNAROUND_TIME"].dt.total_seconds() / 3600, bins=1000, edgecolor="black", alpha=0.75)
plt.title("Distribution of Tasks Turnaround Times")
plt.xlabel("Turnaround Time (Hours)")
plt.ylabel("Count of Tasks")
plt.xlim(0, (df_data_filtered["TURNAROUND_TIME"].dt.total_seconds() / 3600).quantile(0.95))
plt.tight_layout()
plt.show()

In [ ]:
# Calculate turnaround time and hours safely
for col in ["ASSIGNED", "DELIVERED"]:
    if df_data[col].dtype != "datetime64[ns]":
        df_data[col] = pd.to_datetime(df_data[col], errors="coerce")

# Create turnaround time column
mask = (~df_data["ASSIGNED"].isna() & ~df_data["DELIVERED"].isna())
df_data.loc[mask, "TURNAROUND_TIME"] = df_data.loc[mask, "DELIVERED"] - df_data.loc[mask, "ASSIGNED"]

# Filter for reasonable values (positive turnaround time)
reasonable_mask = mask & (df_data["TURNAROUND_TIME"].dt.total_seconds() > 0)

# Calculate hours
df_data.loc[reasonable_mask, "TURNAROUND_HOURS"] = df_data.loc[reasonable_mask, "TURNAROUND_TIME"].dt.total_seconds() / 3600

# Calculate description turnaround time by translator
df_translator_turnaround = df_data.loc[reasonable_mask].groupby("TRANSLATOR")["TURNAROUND_HOURS"].agg(
    ["median", "mean", "std", "count"]
).reset_index()

df_translator_turnaround = df_translator_turnaround.rename(columns={
    "median": "MEDIAN_TURNAROUND",
    "mean": "MEAN_TURNAROUND",
    "std": "STD_TURNAROUND",
    "count": "TASK_COUNT"
})

# Filter to translators with at least 5 tasks
df_translator_turnaround = df_translator_turnaround[df_translator_turnaround["TASK_COUNT"] >= 5]

print("Top translators by median turnaround time (fastest first):")
display(df_translator_turnaround.sort_values("MEDIAN_TURNAROUND").head(20))

In [ ]:
data = df_data.dropna(subset=["COST"])
task_types = data["TASK_TYPE"].unique()
box_data = [data[data["TASK_TYPE"] == task]["COST"] for task in task_types]

# Create the boxplot
plt.figure(figsize=(14, 8))
plt.boxplot(box_data, 
            tick_labels=task_types,
            patch_artist=True, # Enables filling the boxes
            boxprops=dict(alpha=0.75),  # Set color and transparency
            medianprops=dict(color="black"))

plt.title("Cost Distribution by Task Type")
plt.xlabel("Task Type")
plt.ylabel("Cost")
plt.ylim(0, 200)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze quality by task type
data=df_data.dropna(subset=["QUALITY_EVALUATION"])
task_types = data["TASK_TYPE"].unique()
box_data = [data[data["TASK_TYPE"] == task]["QUALITY_EVALUATION"] for task in task_types]

plt.figure(figsize=(14, 8))
plt.boxplot(box_data, 
            tick_labels=task_types,
            patch_artist=True, # Enables filling the boxes
            boxprops=dict(alpha=0.75),  # Set color and transparency
            medianprops=dict(color="black"))
plt.title("Quality Evaluation by Task Type")
plt.xlabel("Task Type")
plt.ylabel("Quality Evaluation")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Comprehensive Language Pair Analysis
df_data["LANGUAGE_PAIR"] = df_data["SOURCE_LANG"] + " > " + df_data["TARGET_LANG"]
language_pair_counts = df_data["LANGUAGE_PAIR"].value_counts()

top_language_pairs = language_pair_counts[:20] # Limited
top_language_pairs_percentage = top_language_pairs / len(df_data) * 100

print("Top Language Pairs:")
for i, (pair, count) in enumerate(top_language_pairs.items()):
    percentage = top_language_pairs_percentage[pair]
    print(f"{i+1}. {pair}: {count} tasks ({percentage:.2f}%)")

# Visualize Top Language Pairs
plt.figure(figsize=(16, 10))
ax = sns.barplot(x=top_language_pairs.values, y=top_language_pairs.index, alpha=0.75)


# Add only percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / df_data.shape[0]
    ax.text(width + 125, p.get_y() + p.get_height()/2, f"{percentage:.1f}%", ha="left", va="center")

plt.title("Top Language Pairs")
plt.xlabel("Count")
plt.ylabel("Language Pair")
plt.grid(True, axis="x", linestyle="--", alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze quality by language pair
top_pairs_lang = top_language_pairs.index
quality_by_pair = df_data[df_data["LANGUAGE_PAIR"].isin(top_pairs_lang)].dropna(subset=["QUALITY_EVALUATION"])

plt.figure(figsize=(16, 10))
ax = sns.boxplot(x="LANGUAGE_PAIR", y="QUALITY_EVALUATION", 
                data=quality_by_pair, 
                order=top_pairs_lang,
                boxprops=dict(alpha=0.75)
)
plt.title("Quality Evaluation by Top Language Pairs")
plt.xlabel("Language Pair")
plt.ylabel("Quality Evaluation")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze cost by language pair
cost_by_pair = df_data[df_data["LANGUAGE_PAIR"].isin(top_pairs_lang)].dropna(subset=["COST"])

plt.figure(figsize=(16, 10))
ax = sns.boxplot(x="LANGUAGE_PAIR", y="COST",
                data=cost_by_pair,
                order=top_pairs_lang,
                boxprops=dict(alpha=0.75)
)
plt.title("Cost Distribution by Top Language Pairs")
plt.xlabel("Language Pair")
plt.ylabel("Cost")
plt.ylim(0, 200)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze hourly rate by language pair
hourly_by_pair = df_data[df_data["LANGUAGE_PAIR"].isin(top_pairs_lang)].dropna(subset=["HOURLY_RATE"])

plt.figure(figsize=(16, 10))
ax = sns.boxplot(x="LANGUAGE_PAIR", y="HOURLY_RATE", 
                data=hourly_by_pair,
                order=top_pairs_lang,
                boxprops=dict(alpha=0.75)
)
plt.title("Hourly Rate Distribution by Top Language Pairs")
plt.xlabel("Language Pair")
plt.ylabel("Hourly Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze industry sectors
sector_counts = df_data["MANUFACTURER_SECTOR"].value_counts()
top_sectors = sector_counts.head(15)
top_sectors_percentage = top_sectors / len(df_data) * 100

print("Top Manufacturer Sectors:")
for i, (sector, count) in enumerate(top_sectors.items()):
    percentage = top_sectors_percentage[sector]
    print(f"{i+1}. {sector}: {count} tasks ({percentage:.2f}%)")

# Visualize Top Sectors
plt.figure(figsize=(16, 10))
ax = sns.barplot(x=top_sectors.values, y=top_sectors.index, alpha=0.75)

# Add count and percentage labels
for i, p in enumerate(ax.patches):
    width = p.get_width()
    percentage = 100 * width / df_data.shape[0]
    ax.text(width + 125, p.get_y() + p.get_height()/2, f"{percentage:.1f}%", ha="left", va="center")

plt.title("Top Manufacturer Sectors")
plt.xlabel("Count")
plt.ylabel("Manufacturer Sector")
plt.grid(True, axis="x", linestyle="--", alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze task duration and workflow in different stages
df_data["ASSIGNMENT_TO_READY"] = (df_data["READY"] - df_data["ASSIGNED"]).dt.total_seconds() / 3600
df_data["READY_TO_WORKING"] = (df_data["WORKING"] - df_data["READY"]).dt.total_seconds() / 3600
df_data["WORKING_TO_DELIVERED"] = (df_data["DELIVERED"] - df_data["WORKING"]).dt.total_seconds() / 3600
df_data["DELIVERED_TO_RECEIVED"] = (df_data["RECEIVED"] - df_data["DELIVERED"]).dt.total_seconds() / 3600
df_data["RECEIVED_TO_CLOSE"] = (df_data["CLOSE"] - df_data["RECEIVED"]).dt.total_seconds() / 3600
df_data["TOTAL_DURATION"] = (df_data["CLOSE"] - df_data["ASSIGNED"]).dt.total_seconds() / 3600

# Filter out negative durations or extreme outliers
duration_cols = [
    "ASSIGNMENT_TO_READY", 
    "READY_TO_WORKING", 
    "WORKING_TO_DELIVERED", 
    "DELIVERED_TO_RECEIVED",
    "RECEIVED_TO_CLOSE",
    "TOTAL_DURATION"
]

for col in duration_cols:
    df_data = df_data[(df_data[col] >= 0) | (df_data[col].isnull())]
    upper_limit = df_data[col].quantile(0.99)
    df_data.loc[df_data[col] > upper_limit, col] = np.nan
    # Obtain number of filtered

# Summary statistics for workflow durations
print("Workflow Duration Statistics (hours):")
duration_stats = df_data[duration_cols].describe().T
duration_stats["% of Total"] = (duration_stats["mean"] / df_data["TOTAL_DURATION"].mean()) * 100
display(duration_stats)

# Visualize workflow stage durations
plt.figure(figsize=(14, 8))
workflow_means = df_data[duration_cols[:-1]].mean()
workflow_means = workflow_means.reindex(duration_cols[:-1])  # Ensure correct order

ax = sns.barplot(x=workflow_means.index, y=workflow_means.values, edgecolor="black", alpha=0.75)
plt.title("Average Duration by Workflow Stage")
plt.xlabel("Workflow Stage")
plt.ylabel("Average Duration (hours)")
plt.xticks(rotation=45)
plt.grid(True, axis="y", linestyle="--", alpha=0.75)
plt.xticks(rotation=45)
# Add duration labels
for i, p in enumerate(ax.patches):
    height = p.get_height()
    percentage = 100 * height / duration_stats.loc["TOTAL_DURATION", "mean"]
    ax.text(p.get_x() + p.get_width()/2, height + 0.1, f"{percentage:.1f}%", ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Analyze workflow duration by task type
workflow_by_task = df_data.groupby("TASK_TYPE")[duration_cols].mean()
print("Workflow Duration by Task Type (hours):")
display(workflow_by_task)

# Visualize working time by task type
plt.figure(figsize=(14, 8))
ax = sns.barplot(x=workflow_by_task.index, y=workflow_by_task["WORKING_TO_DELIVERED"], edgecolor="black", alpha=0.75)
plt.title("Average Working Time by Task Type")
plt.xlabel("Task Type")
plt.ylabel("Average Working Time (hours)")
plt.xticks(rotation=45)
plt.grid(True, axis="y", linestyle="--", alpha=0.75)

# Add duration labels
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2, height + 0.1, f"{height:.1f}h", ha="center", va="bottom")

plt.tight_layout()
plt.show()

In [ ]:
# Analyze translator workload and performance
translator_task_counts = df_data["TRANSLATOR"].value_counts()
translator_stats = df_data.groupby("TRANSLATOR").agg({
    "TASK_ID": "count",
    "QUALITY_EVALUATION": ["mean", "std", "count"],
    "COST": ["mean", "sum"],
    "WORKING_TO_DELIVERED": ["mean", "std"],
    "LANGUAGE_PAIR": lambda x: x.nunique()
}).round(2)

translator_stats.columns = [
    "Task_Count", 
    "Avg_Quality", "Quality_Std", "Quality_Count", 
    "Avg_Cost", "Total_Cost",
    "Avg_Working_Time", "Working_Time_Std",
    "Language_Pair_Count"
]

translator_stats = translator_stats.sort_values("Task_Count", ascending=False)

print("Top Translator Statistics:")
display(translator_stats.head(20))

# Visualize translator task counts
plt.figure(figsize=(16, 10))
ax = sns.barplot(x=translator_task_counts.head(20).values, y=translator_task_counts.head(20).index, edgecolor="black", alpha=0.75)
plt.title("Top Translators by Number of Tasks")
plt.xlabel("Translator")
plt.ylabel("Number of Tasks")
plt.tight_layout()
plt.show()

In [ ]:
# Analyze relationship between task count and quality
plt.figure(figsize=(14, 8))
plt.scatter(
    translator_stats["Task_Count"], 
    translator_stats["Avg_Quality"], 
    alpha=0.35,
    s=translator_stats["Quality_Count"]  # Size based on number of quality evaluations
)
plt.title("Relationship Between Task Count and Average Quality")
plt.xlabel("Number of Tasks")
plt.ylabel("Average Quality Evaluation")
plt.tight_layout()
plt.show()

In [ ]:
# Analyze top translators by language pair
top_translators = translator_stats.head(10).index
language_pairs_by_translator = df_data[df_data["TRANSLATOR"].isin(top_translators)].groupby(
    ["TRANSLATOR", "LANGUAGE_PAIR"]
).size().reset_index(name="count")

# For each top translator, show their three top language pairs
top_translator_pairs = []
for translator in top_translators:
    translator_pairs = language_pairs_by_translator[language_pairs_by_translator["TRANSLATOR"] == translator]
    translator_pairs = translator_pairs.sort_values("count", ascending=False).head(3)
    for _, row in translator_pairs.iterrows():
        top_translator_pairs.append({
            "TRANSLATOR": row["TRANSLATOR"],
            "LANGUAGE_PAIR": row["LANGUAGE_PAIR"],
            "COUNT": row["count"]
        })

translator_top_pairs = pd.DataFrame(top_translator_pairs)
print("Top Language Pairs for Top Translators:")
display(translator_top_pairs)

# Language pair distribution for each language pair
unique_language_pairs = translator_top_pairs["LANGUAGE_PAIR"].unique()

for language_pair in unique_language_pairs:
    data = translator_top_pairs[translator_top_pairs["LANGUAGE_PAIR"] == language_pair]
    
    # Sort data by COUNT to ensure color mapping makes sense
    data = data.sort_values("COUNT")
    
    # Color palette based on the COUNT values
    norm = plt.Normalize(data["COUNT"].min(), data["COUNT"].max())
    colors = plt.cm.YlGnBu(norm(data["COUNT"]))
    
    plt.figure(figsize=(14, 8))
    plt.barh(data["TRANSLATOR"], data["COUNT"], color=colors, edgecolor="black", alpha=0.75)
    plt.title(f"Top Translators for Language Pair: {language_pair}")
    plt.xlabel("Number of Tasks")
    plt.ylabel("Translator")
    plt.grid(True, axis="x", linestyle="--", alpha=0.75)
    plt.tight_layout()
    plt.show()

In [ ]:
# For each sector, identify the most commonly assigned translators
sector_translator = df_data.groupby(["MANUFACTURER_SECTOR", "TRANSLATOR"]).size().reset_index(name="count")
top_sector_translators = []

for sector in top_sectors.index[:5]:  # For top sectors
    sector_data = sector_translator[sector_translator["MANUFACTURER_SECTOR"] == sector]
    sector_data = sector_data.sort_values("count", ascending=False).head(5)
    for _, row in sector_data.iterrows():
        top_sector_translators.append({
            "SECTOR": row["MANUFACTURER_SECTOR"],
            "TRANSLATOR": row["TRANSLATOR"],
            "COUNT": row["count"]
        })

sector_top_translators = pd.DataFrame(top_sector_translators)
print("Top Translators for Top Sectors:")
display(sector_top_translators)


# Separate plots for each sector
for sector in sector_top_translators["SECTOR"].unique():
    sector_data = sector_top_translators[sector_top_translators["SECTOR"] == sector].sort_values("COUNT", ascending=True)
    
    # Color palette based on the COUNT values
    norm = plt.Normalize(sector_data["COUNT"].min(), sector_data["COUNT"].max())
    colors = plt.cm.YlGnBu(norm(sector_data["COUNT"]))
    
    plt.figure(figsize=(16, 8))
    plt.barh(sector_data["TRANSLATOR"], sector_data["COUNT"], color=colors, edgecolor="black", alpha=0.75)
    plt.title(f"Top Translators in {sector} Sector")
    plt.xlabel("Number of Tasks")
    plt.ylabel("Translator")
    plt.grid(True, axis="x", linestyle="--", alpha=0.75)
    plt.tight_layout()
    plt.show()


In [ ]:
# Analyze relationship between cost and quality
quality_cost_df = df_data.dropna(subset=["COST", "QUALITY_EVALUATION"])

plt.figure(figsize=(12, 8))
sns.scatterplot(x="COST", y="QUALITY_EVALUATION", data=quality_cost_df, alpha=0.5)
plt.title("Relationship Between Cost and Quality Evaluation")
plt.xlabel("Cost")
plt.ylabel("Quality Evaluation")
plt.xlim(0, quality_cost_df["COST"].quantile(0.99))  # Exclude extreme outliers
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate correlation between cost and quality
cost_quality_corr = quality_cost_df[["COST", "QUALITY_EVALUATION"]].corr().iloc[0, 1]
print(f"Correlation between Cost and Quality Evaluation: {cost_quality_corr:.4f}")

# Analyze relationship between hourly rate and quality
hourly_quality_df = df_data.dropna(subset=["HOURLY_RATE", "QUALITY_EVALUATION"])

plt.figure(figsize=(12, 8))
sns.scatterplot(x="HOURLY_RATE", y="QUALITY_EVALUATION", data=hourly_quality_df, alpha=0.5)
plt.title("Relationship Between Hourly Rate and Quality Evaluation")
plt.xlabel("Hourly Rate")
plt.ylabel("Quality Evaluation")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate correlation between hourly rate and quality
hourly_quality_corr = hourly_quality_df[["HOURLY_RATE", "QUALITY_EVALUATION"]].corr().iloc[0, 1]
print(f"Correlation between Hourly Rate and Quality Evaluation: {hourly_quality_corr:.4f}")

In [ ]:
df_data["PLANNED_DURATION"] = (df_data["END"] - df_data["START"]).dt.total_seconds() / 3600
df_data["PLANNED_VS_ACTUAL"] = df_data["PLANNED_DURATION"] - df_data["WORKING_TO_DELIVERED"]
df_data["ON_TIME"] = np.where(df_data["PLANNED_VS_ACTUAL"] >= 0, "Yes", "No")

on_time_counts = df_data["ON_TIME"].value_counts()
on_time_percentage = on_time_counts / on_time_counts.sum() * 100

print("\nTask Completion Analysis:")
for status, count in on_time_counts.items():
    percentage = on_time_percentage[status]
    print(f"- {status}: {count} tasks ({percentage:.2f}%)")

# Visualize on-time performance by task type
on_time_by_task = pd.crosstab(df_data["TASK_TYPE"], df_data["ON_TIME"], normalize="index") * 100

plt.figure(figsize=(14, 8))
on_time_by_task["Yes"].sort_values(ascending=False).plot(kind="bar", alpha=0.75)
plt.title("On-Time Completion Percentage by Task Type")
plt.xlabel("Task Type")
plt.ylabel("On-Time Percentage (%)")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualize on-time performance by language pair (top)
on_time_by_pair = df_data[df_data["LANGUAGE_PAIR"].isin(top_pairs_lang)]
on_time_by_pair = pd.crosstab(on_time_by_pair["LANGUAGE_PAIR"], on_time_by_pair["ON_TIME"], normalize="index") * 100

plt.figure(figsize=(14, 8))
on_time_by_pair["Yes"].sort_values(ascending=False).plot(kind="barh", alpha=0.75)
plt.title("On-Time Completion Percentage by Top Language Pairs")
plt.xlabel("Language Pair")
plt.ylabel("On-Time Percentage (%)")
plt.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
print("KEY FINDINGS FROM SAMPLE ANALYSIS:")
print()
print("Tasks and Projects:")
print(f"    {df_data.shape[0]} tasks across {df_data['PROJECT_ID'].nunique()} projects")
print()
print("Translator Information:")
print(f"    {df_data['TRANSLATOR'].nunique()} unique translators")
print(f"    {translator_task_counts.mean():.2f} tasks per translator on average")
print()
print("Task Types:")
print(f"    Most common: '{task_type_counts.index[0]}' ({task_type_percentage[task_type_counts.index[0]]:.2f}% of all tasks)")
print()
print("Language Pairs:")
print(f"    Most common: {language_pair_counts.index[0]} ({top_language_pairs_percentage[language_pair_counts.index[0]]:.2f}% of all tasks)")
print()
print("Quality Metrics:")
print(f"    Average quality evaluation: {df_data['QUALITY_EVALUATION'].mean():.2f}/10")
print()
print("Financial Metrics:")
print(f"    Average task cost: {df_data['COST'].mean():.2f}€")
print(f"    Average hourly rate: {df_data['HOURLY_RATE'].mean():.2f}€")
print()
print("Workflow Analysis:")
print(f"    Most time-consuming stage: '{duration_stats.iloc[duration_stats['mean'].argmax()].name}' ({duration_stats['mean'].max():.2f} hours)")
print()
print("Performance Metrics:")
print(f"    On-time completion rate: {on_time_percentage['Yes']:.2f}%")
print()
print("Correlation Analysis:")
print(f"    Cost and quality correlation: {cost_quality_corr:.4f}")
print(f"    Hourly rate and quality correlation: {hourly_quality_corr:.4f}")
print()
print("Industry Information:")
print(f"    Top industry sector: '{sector_counts.index[0]}' ({top_sectors_percentage[sector_counts.index[0]]:.2f}% of all tasks)")